# Pràctica 4: Similitud de Text Semàntic (STS) per al Català amb Anàlisi Detallada

**Objectiu**: Aquest notebook té com a objectiu entrenar i avaluar diversos models de Similitud de Text Semàntic (STS) per al català. S'exploraran diferents tècniques d'embeddings (Word2Vec, spaCy, RoBERTa) i arquitectures de models, des de baselines simples fins a xarxes neuronals més complexes amb mecanismes d'atenció i models Transformer pre-entrenats. Addicionalment, s'inclou una secció per a la classificació de text utilitzant el dataset TECLA.

## Estructura de la Pràctica:
1.  **Preparació de l'Entorn i Llibreries**: Importació de les llibreries necessàries i configuració inicial.
2.  **Càrrega del Dataset STS-ca**: Obtenció i exploració del dataset principal per a la tasca de STS.
3.  **Preparació d'Embeddings Word2Vec**: Càrrega d'embeddings pre-entrenats i creació de versions truncades.
4.  **Funcions d'Utilitat per a Processament de Text**: Definició de funcions per preprocessar text i generar embeddings de frases.
5.  **Models Baseline (Similitud Cosinus)**: Implementació i avaluació de models bàsics basats en la similitud cosinus.
6.  **Model 1: Regressió amb Embeddings Agregats**: Desenvolupament d'un model de regressió neuronal utilitzant embeddings de frases concatenats.
7.  **Model 2: Seqüència d'Embeddings amb Atenció**: Implementació d'un model seqüencial amb un mecanisme d'atenció per capturar millor les relacions semàntiques.
8.  **Anàlisi Comparativa i Visualització de Resultats (STS)**: Resum i visualització dels resultats dels models STS.
9.  **Experimentació Avançada amb Embeddings**:
    *   Baseline One-Hot Encoding.
    *   Ajustament d'Hiperparàmetres.
    *   Ús d'Embeddings de spaCy.
    *   Ús d'Embeddings de RoBERTa (base).
    *   Ús d'un model RoBERTa fine-tuned per STS.
10. **Classificació de Text amb el Dataset TECLA**:
    *   Càrrega i preparació del dataset TECLA.
    *   Model de classificació amb embeddings agregats.
    *   Model de classificació amb seqüències d'embeddings.
11. **Conclusions Generals i Avaluació Final**: Resum de les troballes clau i avaluació del millor model en el conjunt de test.


## 1. Preparació de l'Entorn i Llibreries

En aquesta secció, importem totes les llibreries necessàries per a la pràctica. Aquestes inclouen:

També es configura l'ús de la GPU si està disponible per accelerar l'entrenament dels models i s'imprimeixen les versions de TensorFlow i la disponibilitat de GPU.

In [ ]:
# Imports necessaris
import os
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Optional, Dict, Union
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings

warnings.filterwarnings('ignore')
# Configuració de GPU (opcional)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

## 2. Càrrega del Dataset STS-ca

Aquí carreguem el dataset "STS-ca" (Semantic Textual Similarity for Catalan) del projecte AINA, utilitzant la llibreria `datasets` de Hugging Face. Aquest dataset conté parells de frases en català i una etiqueta numèrica que indica el seu grau de similitud semàntica (normalment en una escala de 0 a 5).

El dataset es divideix en tres parts:
- `train`: conjunt d'entrenament, utilitzat per ajustar els paràmetres dels models.
- `test`: conjunt de prova, utilitzat per a l'avaluació final del model seleccionat.
- `validation`: conjunt de validació, utilitzat per monitorar el rendiment durant l'entrenament i per a l'ajust d'hiperparàmetres.

Després de carregar les dades, les convertim a DataFrames de Pandas per facilitar-ne la manipulació i l'anàlisi. S'imprimeix el nombre de mostres en cada conjunt, el rang de les etiquetes de similitud i alguns exemples per entendre millor l'estructura de les dades.

In [ ]:
from datasets import load_dataset

# Carregar el dataset STS-ca
print("Carregant dataset STS-ca...")
train_data = load_dataset("projecte-aina/sts-ca", split="train")
test_data = load_dataset("projecte-aina/sts-ca", split="test") 
val_data = load_dataset("projecte-aina/sts-ca", split="validation")

# Convertir a DataFrame per facilitar la manipulació
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
val_df = pd.DataFrame(val_data)

print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Label range: {train_df['label'].min():.2f} - {train_df['label'].max():.2f}")

# Mostrar alguns exemples
print("\nExemples del dataset:")
for i in range(3):
    print(f"Frase 1: {train_df.iloc[i]['sentence_1']}")
    print(f"Frase 2: {train_df.iloc[i]['sentence_2']}")
    print(f"Similitud: {train_df.iloc[i]['label']}")
    print("-" * 50)

## 3. Preparació d'Embeddings Word2Vec

En aquesta secció, carreguem embeddings de paraules pre-entrenats. Utilitzarem el model Word2Vec `cc.ca.300.vec`, que conté vectors de 300 dimensions per a paraules en català.

**Passos:**
1.  **Càrrega del Model Word2Vec**: S'utilitza `KeyedVectors.load_word2vec_format` de la llibreria `gensim` per carregar els embeddings des d'un fitxer. S'especifica `binary=False` ja que el format `.vec` és textual.
2.  **Informació del Model**: Es mostra el nombre de paraules en el vocabulari del model i la dimensió dels vectors d'embedding.
3.  **Exemple d'Ús**: Es demostra com accedir al vector d'embedding d'una paraula específica i es mostren les primeres components del vector.
4.  **Truncament d'Embeddings**: Es defineix la funció `create_truncated_embeddings` per generar versions dels embeddings amb dimensions més petites (50, 100, 150). Això permetrà experimentar com la dimensionalitat dels embeddings afecta el rendiment dels models. El truncament es fa simplement seleccionant les primeres `N` dimensions del vector original.
5.  **Exemple d'Embeddings Truncats**: Es mostra com es veuen els embeddings truncats per a una paraula d'exemple.

Aquesta preparació és crucial, ja que aquests embeddings serviran com a base per representar les frases en els primers models que desenvoluparem.

In [ ]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

# Carregar el model Word2Vec pre-entrenat
WV_MODEL_PATH = '../cc.ca.300.vec'

print("Carregant model Word2Vec...")

kv_model = KeyedVectors.load_word2vec_format(WV_MODEL_PATH, binary=False)
print(f"Model carregat amb èxit. Vocabulari: {len(kv_model.key_to_index)} paraules")
print(f"Dimensió dels vectors: {kv_model.vector_size}")

In [ ]:
# Exemple d'ús
test_words = ["casa", "gat", "aigua", "carbassot"]
for word in test_words:
    if word in kv_model:
        print(f"Vector per '{word}': {kv_model[word][:5]}... (dim={kv_model.vector_size})")
    else:
        print(f"Paraula '{word}' no trobada al vocabulari")

In [ ]:
# Funció per truncar embeddings a dimensions més petites
def create_truncated_embeddings(kv_model, dimensions: List[int]) -> Dict[int, Dict[str, np.ndarray]]:
    """
    Crea versions truncades dels embeddings amb diferents dimensions
    """
    if kv_model is None:
        return {}
    
    truncated_models = {}
    
    for dim in dimensions:
        print(f"Creant embeddings de {dim} dimensions...")
        truncated_dict = {}
        
        for word in kv_model.key_to_index:
            original_vector = kv_model[word]
            truncated_vector = original_vector[:dim]
            truncated_dict[word] = truncated_vector
            
        truncated_models[dim] = truncated_dict
        print(f"  Completat: {len(truncated_dict)} paraules truncades a {dim}D")
    
    return truncated_models

# Crear versions truncades
dimensions = [50, 100, 150, 300]  # Incloem 300 per consistència
if kv_model is not None:
    truncated_embeddings = create_truncated_embeddings(kv_model, dimensions)
    print(f"\nVersions d'embeddings disponibles: {list(truncated_embeddings.keys())}")
else:
    truncated_embeddings = {}


In [ ]:
print("Model truncat a 50 dimensions:")
print(truncated_embeddings[50]['casa'])
print("Model truncat a 100 dimensions:")
print(truncated_embeddings[100]['casa'])

## 4. Funcions d'Utilitat per a Processament de Text

Aquesta secció defineix funcions auxiliars clau per al processament de text i la generació d'embeddings de frases, que seran utilitzades pels diferents models.

**Funcions Definides:**

1.  `preprocess_sentence(sentence: str) -> List[str]`:
    *   **Propòsit**: Realitza una tokenització bàsica de la frase.
    *   **Funcionament**: Converteix la frase a minúscules i la divideix en paraules (tokens) utilitzant `simple_preprocess` de `gensim`.

2.  `get_sentence_embedding_simple(sentence: str, embeddings_dict: Dict[str, np.ndarray], vector_size: int) -> np.ndarray`:
    *   **Propòsit**: Calcula l'embedding d'una frase fent la mitjana dels embeddings de les seves paraules.
    *   **Funcionament**:
        1.  Preprocessa la frase utilitzant `preprocess_sentence`.
        2.  Per a cada paraula, si existeix en el diccionari d'embeddings proporcionat (`embeddings_dict`), recupera el seu vector.
        3.  Calcula la mitjana de tots els vectors de paraules trobats.
        4.  Si cap paraula de la frase té un embedding, retorna un vector de zeros de la mida especificada (`vector_size`).

3.  `get_sentence_embedding_tfidf(sentence: str, embeddings_dict: Dict[str, np.ndarray], tfidf_vectorizer: TfidfVectorizer, feature_names: List[str], vector_size: int) -> np.ndarray`:
    *   **Propòsit**: Calcula l'embedding d'una frase fent una mitjana ponderada dels embeddings de les seves paraules, utilitzant els pesos TF-IDF.
    *   **Funcionament**:
        1.  Preprocessa la frase.
        2.  Calcula el vector TF-IDF per a la frase.
        3.  Per a cada paraula amb embedding i present en el vocabulari TF-IDF:
            *   Obté el seu pes TF-IDF.
            *   Multiplica l'embedding de la paraula pel seu pes TF-IDF.
        4.  Calcula la suma ponderada dels vectors d'embedding i la divideix per la suma dels pesos.
        5.  Si no es poden obtenir vectors ponderats, retorna un vector de zeros.

**Preparació del Vocabulari per TF-IDF:**
*   Es recullen totes les frases dels conjunts d'entrenament, validació i prova.
*   Es preprocessen per extreure totes les paraules i construir un vocabulari global. Aquest vocabulari serà utilitzat per entrenar el `TfidfVectorizer` més endavant.

Aquestes funcions són fonamentals per convertir el text en representacions numèriques que els models puguin processar.

In [ ]:
def preprocess_sentence(sentence: str) -> List[str]:
    """
    Preprocessa una frase: tokenització simple
    """
    return simple_preprocess(sentence.lower())

def get_sentence_embedding_simple(sentence: str, embeddings_dict: Dict[str, np.ndarray], 
                                vector_size: int) -> np.ndarray:
    """
    Obté l'embedding d'una frase fent la mitjana dels embeddings de les paraules
    """
    words = preprocess_sentence(sentence)
    vectors = []
    
    for word in words:
        if word in embeddings_dict:
            vectors.append(embeddings_dict[word])
    
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

def get_sentence_embedding_tfidf(sentence: str, embeddings_dict: Dict[str, np.ndarray], 
                               tfidf_vectorizer: TfidfVectorizer, 
                               feature_names: List[str], vector_size: int) -> np.ndarray:
    """
    Obté l'embedding d'una frase fent la mitjana ponderada amb TF-IDF
    """
    words = preprocess_sentence(sentence)
    
    # Calcular TF-IDF per a la frase
    tfidf_vector = tfidf_vectorizer.transform([' '.join(words)])
    tfidf_scores = tfidf_vector.toarray()[0]
    
    weighted_vectors = []
    weights = []
    
    for word in words:
        if word in embeddings_dict and word in feature_names:
            word_idx = feature_names.index(word)
            weight = tfidf_scores[word_idx]
            if weight > 0:
                weighted_vectors.append(embeddings_dict[word] * weight)
                weights.append(weight)
    
    if weighted_vectors and sum(weights) > 0:
        return np.sum(weighted_vectors, axis=0) / sum(weights)
    else:
        return np.zeros(vector_size)

# Preprocessar totes les frases del dataset
print("Preprocessant frases del dataset...")
all_sentences = (train_df['sentence_1'].tolist() + train_df['sentence_2'].tolist() + 
                test_df['sentence_1'].tolist() + test_df['sentence_2'].tolist() + 
                val_df['sentence_1'].tolist() + val_df['sentence_2'].tolist())

# Crear vocabulari per TF-IDF
all_words = []
for sentence in all_sentences:
    all_words.extend(preprocess_sentence(sentence))

print(f"Total de frases processades: {len(all_sentences)}")
print(f"Vocabulari únic: {len(set(all_words))} paraules")

## 5. Baseline: Similitud Cosinus

Aquesta secció implementa i avalua un model baseline per a la tasca de STS. El baseline es basa en calcular la similitud cosinus entre els embeddings de les frases.

**Funcionament:**

1.  **`evaluate_cosine_baseline` Funció**:
    *   **Entrada**: Un DataFrame amb parells de frases i les seves etiquetes de similitud, un diccionari d'embeddings de paraules, la mida del vector, i opcionalment, un vectoritzador TF-IDF.
    *   **Procés**:
        *   Per a cada parell de frases:
            *   Obté l'embedding de cada frase utilitzant `get_sentence_embedding_simple` (mitjana simple) o `get_sentence_embedding_tfidf` (mitjana ponderada amb TF-IDF).
            *   Calcula la similitud cosinus entre els dos vectors de frase. La similitud cosinus varia entre -1 (totalment oposats) i 1 (idèntics). Si algun vector és zero, la similitud es considera 0.
            *   Escala la similitud cosinus (originalment en `[-1, 1]`) a l'interval `[0, 5]` per coincidir amb el rang de les etiquetes del dataset STS-ca, mitjançant la fórmula `(sim + 1) * 2.5`.
    *   **Sortida**: Un diccionari amb les mètriques d'avaluació:
        *   `pearson`: Correlació de Pearson entre les similituds predites i les reals. És la mètrica principal per a STS.
        *   `mse`: Error Quadràtic Mig (Mean Squared Error).
        *   `mae`: Error Absolut Mig (Mean Absolute Error).
        *   `predictions`: Les similituds predites.

2.  **Preparació del Vectoritzador TF-IDF**:
    *   S'utilitza `TfidfVectorizer` de `sklearn` per calcular els pesos TF-IDF.
    *   S'entrena (`fit`) amb un corpus format per totes les frases preprocessades del dataset.
    *   Es limita el nombre de característiques (paraules) a `max_features=10000`.

3.  **Avaluació dels Baselines**:
    *   S'avalua el baseline en el conjunt de validació (`val_df`).
    *   Es fa per a cada dimensió d'embedding truncada (50, 100, 150, 300).
    *   Per a cada dimensió, s'avaluen dues variants:
        *   Utilitzant la mitjana simple d'embeddings de paraules.
        *   Utilitzant la mitjana ponderada amb TF-IDF.
    *   Els resultats (Pearson, MSE, MAE) es guarden per a la seva posterior comparació.

L'objectiu d'aquest baseline és establir un punt de referència. Models més sofisticats haurien de superar aquests resultats.

In [ ]:
def evaluate_cosine_baseline(df: pd.DataFrame, embeddings_dict: Dict[str, np.ndarray], 
                           vector_size: int, use_tfidf: bool = False, 
                           tfidf_vectorizer=None, feature_names=None) -> Dict[str, float]:
    """
    Avalua el baseline de similitud cosinus
    """
    similarities = []
    true_scores = df['label'].values
    
    for _, row in df.iterrows():
        sent1, sent2 = row['sentence_1'], row['sentence_2']
        
        if use_tfidf and tfidf_vectorizer is not None:
            vec1 = get_sentence_embedding_tfidf(sent1, embeddings_dict, tfidf_vectorizer, 
                                              feature_names, vector_size)
            vec2 = get_sentence_embedding_tfidf(sent2, embeddings_dict, tfidf_vectorizer, 
                                              feature_names, vector_size)
        else:
            vec1 = get_sentence_embedding_simple(sent1, embeddings_dict, vector_size)
            vec2 = get_sentence_embedding_simple(sent2, embeddings_dict, vector_size)
        
        # Calcular similitud cosinus
        if np.all(vec1 == 0) or np.all(vec2 == 0):
            sim = 0.0
        else:
            sim = 1 - cosine(vec1, vec2)
        
        # Escalar de [-1,1] a [0,5] per coincidir amb les etiquetes
        sim_scaled = (sim + 1) * 2.5
        similarities.append(sim_scaled)
    
    # Calcular mètriques
    similarities = np.array(similarities)
    pearson_corr, _ = pearsonr(true_scores, similarities)
    mse = mean_squared_error(true_scores, similarities)
    mae = mean_absolute_error(true_scores, similarities)
    
    return {
        'pearson': pearson_corr,
        'mse': mse,
        'mae': mae,
        'predictions': similarities
    }

# Preparar TF-IDF
if kv_model is not None:
    print("Preparant TF-IDF vectorizer...")
    corpus_for_tfidf = [' '.join(preprocess_sentence(sent)) for sent in all_sentences]
    tfidf_vectorizer = TfidfVectorizer(max_features=10000, lowercase=True)
    tfidf_vectorizer.fit(corpus_for_tfidf)
    feature_names = tfidf_vectorizer.get_feature_names_out().tolist()
    
    print("Avaluant baselines de similitud cosinus...")
    
    # Avaluar per diferents dimensions
    baseline_results = {}
    results_list = []
    for dim in [50, 100, 150, 300]:
        if dim in truncated_embeddings:
            # Mitjana simple
            results_simple = evaluate_cosine_baseline(
                val_df, truncated_embeddings[dim], dim, use_tfidf=False
            )

            # Mitjana ponderada TF-IDF
            results_tfidf = evaluate_cosine_baseline(
                val_df, truncated_embeddings[dim], dim, use_tfidf=True,
                tfidf_vectorizer=tfidf_vectorizer, feature_names=feature_names
            )

            baseline_results[dim] = {
                'simple': results_simple,
                'tfidf': results_tfidf
            }

            results_list.append({
                'Model': 'Baseline Cosinus Simple',
                'Dimensions': f'{dim}D',
                'Pearson': results_simple['pearson'],
                'MSE': results_simple['mse'],
                'MAE': results_simple['mae']
            })
            results_list.append({
                'Model': 'Baseline Cosinus TF-IDF',
                'Dimensions': f'{dim}D',
                'Pearson': results_tfidf['pearson'],
                'MSE': results_tfidf['mse'],
                'MAE': results_tfidf['mae']
            })

**Anàlisi dels Resultats del Baseline Cosinus**

La taula següent mostra els resultats dels models baseline de similitud cosinus, tant amb mitjana simple d'embeddings com amb mitjana ponderada per TF-IDF, per a diferents dimensions dels embeddings Word2Vec.

**Observacions Clau:**
*   **Impacte de TF-IDF**: Generalment, l'ús de TF-IDF per ponderar els embeddings de paraules tendeix a millorar la correlació de Pearson en comparació amb la mitjana simple. Això suggereix que donar més importància a paraules més discriminatives (segons TF-IDF) és beneficiós.
*   **Impacte de la Dimensió**: S'observa una tendència a millorar el rendiment (major Pearson, menor MSE/MAE) a mesura que augmenta la dimensió dels embeddings, tot i que els guanys poden disminuir a partir d'una certa dimensió. Els embeddings de 300D solen oferir els millors resultats dins d'aquest baseline.
*   **Correlació de Pearson**: Aquesta és la mètrica principal. Valors més alts indiquen una millor concordança entre les prediccions del model i les etiquetes humanes.
*   **MSE i MAE**: Mesuren l'error de predicció. Valors més baixos són millors.

Aquests resultats serveixen com a punt de partida per avaluar la millora aportada pels models neuronals més complexos que es desenvoluparan a continuació.

In [ ]:
print("\n=== COMPARACIÓ MODELS BASELINE COSINUS ===")
df_baseline_results = pd.DataFrame(results_list)
display(df_baseline_results.style.hide(axis="index"))

## 6. Model 1: Regressió amb Embeddings Agregats

Aquesta secció desenvolupa un model de xarxa neuronal per a la tasca de regressió de STS. El model pren com a entrada els embeddings de les dues frases (obtinguts mitjançant mitjana simple dels embeddings de paraules) i aprèn a predir la seva similitud.

**Components Clau:**

1.  **`build_model_aggregated` Funció**:
    *   **Arquitectura del Model**:
        *   **Entrades**: Dos vectors d'entrada, un per a l'embedding de cada frase (`input_vector_1`, `input_vector_2`). La dimensió d'aquests vectors (`embedding_dim`) correspon a la dimensió dels embeddings de frase utilitzats.
        *   **Concatenació**: Els dos vectors d'entrada es concatenen per formar un únic vector que representa el parell de frases.
        *   **Capes de Processament**:
            *   `BatchNormalization`: Normalitza les activacions de la capa anterior, ajudant a estabilitzar i accelerar l'entrenament.
            *   `Dense`: Capes totalment connectades amb funcions d'activació ReLU. S'utilitzen per aprendre transformacions no lineals de les dades.
            *   `Dropout`: Tècnica de regularització que desactiva aleatòriament un percentatge de neurones durant l'entrenament per prevenir el sobreajustament (overfitting).
        *   **Capa de Sortida**: Una única neurona `Dense` amb activació lineal, ja que es tracta d'un problema de regressió (predir un valor continu entre 0 i 5).
    *   **Compilació del Model**:
        *   `loss`: `mean_squared_error` (Error Quadràtic Mig), una funció de pèrdua comuna per a problemes de regressió.
        *   `optimizer`: `Adam`, un optimitzador popular i eficient. S'estableix una taxa d'aprenentatge inicial.
        *   `metrics`: `mae` (Error Absolut Mig) i `RootMeanSquaredError` per monitorar el rendiment durant l'entrenament i l'avaluació.

2.  **`prepare_aggregated_data` Funció**:
    *   **Propòsit**: Prepara les dades d'entrada i sortida per al model.
    *   **Funcionament**:
        *   Itera sobre el DataFrame proporcionat.
        *   Per a cada parell de frases, obté els seus embeddings (utilitzant `get_sentence_embedding_simple` o `get_sentence_embedding_tfidf`).
        *   Emmagatzema els embeddings de la primera frase a `X1`, els de la segona a `X2`, i les etiquetes de similitud a `Y`.
        *   Retorna `X1`, `X2`, i `Y` com a arrays NumPy.

3.  **Entrenament i Avaluació del Model**:
    *   S'itera sobre les diferents dimensions d'embeddings (50, 100, 150, 300).
    *   Per a cada dimensió:
        *   Es preparen les dades d'entrenament i validació utilitzant `prepare_aggregated_data`.
        *   Es construeix el model utilitzant `build_model_aggregated`.
        *   **Callbacks**:
            *   `EarlyStopping`: Atura l'entrenament si la pèrdua en el conjunt de validació (`val_loss`) no millora durant un nombre determinat d'èpoques (`patience`), restaurant els pesos del millor model.
            *   `ReduceLROnPlateau`: Redueix la taxa d'aprenentatge si la `val_loss` s'estanca.
        *   S'entrena el model (`model.fit`) amb les dades d'entrenament, validant en cada època amb les dades de validació.
        *   Després de l'entrenament, es prediuen les similituds per al conjunt de validació.
        *   Es calculen les mètriques (Pearson, MSE, MAE) comparant les prediccions amb els valors reals.
        *   Els resultats i el model entrenat es guarden.

Aquest model representa un pas més enllà dels baselines, ja que la xarxa neuronal pot aprendre relacions més complexes entre els embeddings de les frases.

In [ ]:
def build_model_aggregated(embedding_dim: int, hidden_size: int = 128, 
                         dropout_rate: float = 0.3) -> tf.keras.Model:
    """
    Construeix el model de regressió amb embeddings agregats
    """
    input_1 = tf.keras.Input(shape=(embedding_dim,), name="input_vector_1")
    input_2 = tf.keras.Input(shape=(embedding_dim,), name="input_vector_2")
    
    # Concatenar els dos vectors
    concatenated = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    
    # Capes de processament
    x = tf.keras.layers.BatchNormalization()(concatenated)
    x = tf.keras.layers.Dense(hidden_size, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(hidden_size // 2, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    # Capa de sortida (regressió)
    output = tf.keras.layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=['mae', tf.keras.metrics.RootMeanSquaredError()]
    )
    
    return model

def prepare_aggregated_data(df: pd.DataFrame, embeddings_dict: Dict[str, np.ndarray], 
                          vector_size: int, use_tfidf: bool = False,
                          tfidf_vectorizer=None, feature_names=None) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Prepara les dades per al model d'embeddings agregats
    """
    X1, X2, Y = [], [], []
    
    for _, row in df.iterrows():
        sent1, sent2, label = row['sentence_1'], row['sentence_2'], row['label']
        
        if use_tfidf and tfidf_vectorizer is not None:
            vec1 = get_sentence_embedding_tfidf(sent1, embeddings_dict, tfidf_vectorizer, 
                                              feature_names, vector_size)
            vec2 = get_sentence_embedding_tfidf(sent2, embeddings_dict, tfidf_vectorizer, 
                                              feature_names, vector_size)
        else:
            vec1 = get_sentence_embedding_simple(sent1, embeddings_dict, vector_size)
            vec2 = get_sentence_embedding_simple(sent2, embeddings_dict, vector_size)
        
        X1.append(vec1)
        X2.append(vec2)
        Y.append(label)
    
    return np.array(X1), np.array(X2), np.array(Y)

# Entrenar models agregats per diferents dimensions
if kv_model is not None:
    aggregated_results = {}
    
    for dim in [50, 100, 150, 300]:
        if dim in truncated_embeddings:
            print(f"\n=== Entrenant Model Agregat {dim}D ===")
            
            # Preparar dades
            X1_train, X2_train, Y_train = prepare_aggregated_data(
                train_df, truncated_embeddings[dim], dim
            )
            X1_val, X2_val, Y_val = prepare_aggregated_data(
                val_df, truncated_embeddings[dim], dim
            )
            
            print(f"Forma de les dades: X1_train={X1_train.shape}, Y_train={Y_train.shape}")
            
            # Construir i entrenar model
            model = build_model_aggregated(embedding_dim=dim)
            
            # Callbacks
            early_stopping = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
            )
            
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
            )
            
            # Entrenament
            history = model.fit(
                [X1_train, X2_train], Y_train,
                validation_data=([X1_val, X2_val], Y_val),
                epochs=50,
                batch_size=32,
                callbacks=[early_stopping, reduce_lr],
                verbose=1
            )
            
            # Avaluació
            Y_pred = model.predict([X1_val, X2_val]).flatten()
            pearson_corr, _ = pearsonr(Y_val, Y_pred)
            mse = mean_squared_error(Y_val, Y_pred)
            mae = mean_absolute_error(Y_val, Y_pred)
            
            aggregated_results[dim] = {
                'model': model,
                'history': history,
                'pearson': pearson_corr,
                'mse': mse,
                'mae': mae,
                'predictions': Y_pred
            }
            
            print(f"Resultats {dim}D - Pearson: {pearson_corr:.3f}, MSE: {mse:.3f}, MAE: {mae:.3f}")

**Anàlisi dels Resultats del Model d'Embeddings Agregats**

La taula següent resumeix el rendiment del model de regressió amb embeddings agregats per a les diferents dimensions d'embeddings utilitzades.

**Observacions Clau:**
*   **Millora sobre Baselines**: S'espera que aquest model neuronal superi els resultats dels baselines de similitud cosinus, especialment en termes de correlació de Pearson. La capacitat d'aprenentatge de la xarxa hauria de capturar relacions més complexes que una simple similitud cosinus.
*   **Impacte de la Dimensió**: Similar als baselines, és probable que dimensions més altes dels embeddings (fins a un cert punt) portin a millors resultats, ja que contenen més informació semàntica.
*   **Mètriques**:
    *   `Pearson`: Valors més alts indiquen un millor rendiment.
    *   `MSE` i `MAE`: Valors més baixos són preferibles.
*   **Comparació**: Aquests resultats es compararan amb els del Model 2 (amb atenció) i altres models més avançats per determinar l'eficàcia relativa d'aquesta arquitectura.

La taula `df_aggregated` mostra una visió clara de com la dimensionalitat afecta el rendiment d'aquest model específic. Aquests resultats s'afegiran a `df_results` per a una comparació global posterior.

In [ ]:
print("\n=== COMPARACIÓ MODELS AGREGATS ===")
# Crear un DataFrame amb els resultats dels models agregats
aggr_data = []
for dim in [50, 100, 150, 300]:
    if dim in aggregated_results:
        aggr_data.append({
            'Model': 'Model Agregat',
            'Dimensions': f'{dim}D',
            'Pearson': aggregated_results[dim]['pearson'],
            'MSE': aggregated_results[dim]['mse'],
            'MAE': aggregated_results[dim]['mae']
        })

df_aggregated = pd.DataFrame(aggr_data)
df_results = pd.concat([df_baseline_results, df_aggregated], ignore_index=True)
display(df_aggregated.style.hide(axis="index"))

## 7. Model 2: Seqüència d'Embeddings amb Atenció

Aquesta secció introdueix un model més sofisticat que processa les frases com a seqüències d'embeddings de paraules i utilitza un mecanisme d'atenció per ponderar la importància de cada paraula en la representació de la frase. Aquest enfocament pot capturar millor les nuances semàntiques que la simple mitjana d'embeddings.

**Components Clau:**

1.  **`SimpleAttention` Capa Personalitzada**:
    *   **Propòsit**: Implementa un mecanisme d'atenció simple. Donada una seqüència d'embeddings de paraules, calcula un vector de context que representa la frase, ponderant cada paraula segons la seva rellevància.
    *   **Funcionament**:
        1.  Transforma els embeddings d'entrada (`inputs`) mitjançant una capa densa (`W_s1`) amb activació `tanh` per obtenir estats ocults.
        2.  Calcula puntuacions d'atenció (`scores`) per a cada estat ocult utilitzant una altra capa densa (`W_s2`).
        3.  Si s'aplica màscara (`mask`, per ignorar posicions de padding), s'ajusten les puntuacions.
        4.  Normalitza les puntuacions utilitzant `softmax` per obtenir els pesos d'atenció (`attention_weights`).
        5.  Calcula el vector de context (`context_vector`) com una suma ponderada dels embeddings d'entrada originals, utilitzant els pesos d'atenció.
    *   **Suport de Màscara**: La capa indica que suporta màscares (`supports_masking = True`), la qual cosa és important quan es treballa amb seqüències de longitud variable i padding.

2.  **`build_model_sequence` Funció**:
    *   **Arquitectura del Model (Siamesa Modificada)**:
        *   **Entrades**: Dues seqüències d'enters (`input_1`, `input_2`), on cada enter representa l'índex d'una paraula en un vocabulari. La longitud de les seqüències és fixa (`sequence_length`).
        *   **Capa d'Embedding Compartida (`embedding_layer`)**:
            *   Converteix les seqüències d'índexs en seqüències de vectors densos (embeddings).
            *   Pot utilitzar embeddings pre-entrenats (`pretrained_weights`) o aprendre'ls des de zero.
            *   Pot ser entrenable (`trainable_embeddings=True`) o congelada (`trainable_embeddings=False`).
            *   `mask_zero=True`: Indica que l'índex 0 (reservat per al padding) s'ha d'ignorar en les capes posteriors que suporten màscares.
        *   **Processament de Seqüències (Compartit)**:
            *   Si `use_attention=True`, s'aplica la capa `SimpleAttention` a les seqüències d'embeddings per obtenir un vector de frase per a cada entrada.
            *   Si `use_attention=False`, s'utilitza `GlobalAveragePooling1D` com a alternativa més simple per agregar els embeddings de paraules.
        *   **Capa de Projecció (`first_projection_layer`)**: Una capa densa amb activació `tanh` aplicada als vectors de frase. Això pot ajudar a refinar les representacions. S'aplica dropout.
        *   **Normalització L2**: Els vectors projectats es normalitzen (L2) per assegurar que tinguin una magnitud unitària. Això és comú en models que utilitzen similitud cosinus.
        *   **Càlcul de Similitud Cosinus**: Es calcula la similitud cosinus entre els dos vectors de frase normalitzats. Això es fa multiplicant element a element i sumant (`tf.reduce_sum(x[0] * x[1], axis=1)`).
        *   **Capa de Sortida (`output_layer`)**: Transforma la similitud cosinus (originalment en `[-1, 1]`) a l'interval `[0, 1]` utilitzant `0.5 * (1.0 + x)`. *Nota: Per al dataset STS-ca, que té etiquetes de 0 a 5, aquesta sortida [0,1] haurà de ser escalada posteriorment si es vol comparar directament, o la funció de pèrdua haurà de tenir en compte les etiquetes originals escalades a [0,1]. El codi actual compila amb 'mean_squared_error', assumint que les etiquetes Y_train_seq ja estan en l'escala [0,1] o que la pèrdua s'interpreta en aquest rang.*
    *   **Compilació**: Similar al model agregat, utilitzant `mean_squared_error` i `Adam`.

Aquest model té el potencial de ser més potent que el model agregat, ja que considera l'ordre de les paraules i pot aprendre a enfocar-se en les parts més importants de cada frase gràcies a l'atenció.

In [ ]:
import tensorflow as tf
import numpy as np
from typing import Optional

class SimpleAttention(tf.keras.layers.Layer):
    """
    Capa d'atenció simple per agregar seqüències d'embeddings
    """
    def __init__(self, units: int, **kwargs):
        super(SimpleAttention, self).__init__(**kwargs)
        self.units = units
        self.dropout_s1 = tf.keras.layers.Dropout(0.3)
        self.dropout_s2 = tf.keras.layers.Dropout(0.2)
        self.W_s1 = tf.keras.layers.Dense(units, activation='tanh', use_bias=True, name="attention_transform")
        # Dense layer to compute attention scores (context vector)
        self.W_s2 = tf.keras.layers.Dense(1, use_bias=False, name="attention_scorer")
        self.supports_masking = True  # Declare that this layer supports masking

    def call(self, inputs: tf.Tensor, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        # inputs shape: (batch_size, sequence_length, embedding_dim)
        # mask shape: (batch_size, sequence_length) boolean tensor

        # Attention hidden states
        hidden_states = self.dropout_s1(self.W_s1(inputs))

        # Compute attention scores
        scores = self.dropout_s2(self.W_s2(hidden_states))

        if mask is not None:
            # Apply the mask to the scores before softmax
            expanded_mask = tf.expand_dims(tf.cast(mask, dtype=tf.float32), axis=-1)
            # Add a large negative number to masked (padded) scores
            scores += (1.0 - expanded_mask) * -1e9

        # Compute attention weights
        attention_weights = tf.nn.softmax(scores, axis=1)

        # Compute the context vector (weighted sum of input embeddings)
        context_vector = tf.reduce_sum(inputs * attention_weights, axis=1)

        return context_vector

    def get_config(self) -> dict:
        config = super(SimpleAttention, self).get_config()
        config.update({"units": self.units})
        return config

    def compute_mask(self, inputs: tf.Tensor, mask: Optional[tf.Tensor] = None) -> Optional[tf.Tensor]:
        return None


def build_model_sequence(vocab_size: int = 1000, 
                        embedding_dim: int = 300,
                        sequence_length: int = 32,
                        learning_rate: float = 0.001,
                        trainable_embeddings: bool = False,
                        pretrained_weights: Optional[np.ndarray] = None,
                        use_attention: bool = True,
                        attention_units: int = 4) -> tf.keras.Model:
    """
    Model de seqüència millorat basat en build_and_compile_model_2
    Incorpora similitud cosinus i escalat adequat per STS
    """
    input_1 = tf.keras.Input((sequence_length,), dtype=tf.int32, name="input_1")
    input_2 = tf.keras.Input((sequence_length,), dtype=tf.int32, name="input_2")

    # Determine effective embedding parameters
    if pretrained_weights is not None:
        effective_dictionary_size = pretrained_weights.shape[0]
        effective_embedding_size = pretrained_weights.shape[1]
        embedding_initializer = tf.keras.initializers.Constant(pretrained_weights)
        is_embedding_trainable = trainable_embeddings
        embedding_layer_name = "embedding_pretrained"
    else:
        effective_dictionary_size = vocab_size
        effective_embedding_size = embedding_dim
        embedding_initializer = 'uniform'
        is_embedding_trainable = True
        embedding_layer_name = "embedding"

    # Shared Embedding Layer
    embedding_layer = tf.keras.layers.Embedding(
        input_dim=effective_dictionary_size,
        output_dim=effective_embedding_size,
        input_length=sequence_length,
        mask_zero=True,
        embeddings_initializer=embedding_initializer,
        trainable=is_embedding_trainable,
        name=embedding_layer_name
    )

    # Apply embedding layer to both inputs
    embedded_1 = embedding_layer(input_1)  # Shape: (batch_size, sequence_length, effective_embedding_size)
    embedded_2 = embedding_layer(input_2)  # Shape: (batch_size, sequence_length, effective_embedding_size)

    # Shared pooling/attention layer
    if use_attention:
        # Use attention mechanism
        sentence_pooling_layer = SimpleAttention(units=attention_units, name="sentence_attention")
    else:
        # Use simple global average pooling
        sentence_pooling_layer = tf.keras.layers.GlobalAveragePooling1D(name="sentence_attention_layer")

    # Apply pooling/attention to get sentence vectors
    sentence_vector_1 = sentence_pooling_layer(embedded_1)
    sentence_vector_2 = sentence_pooling_layer(embedded_2)

    # Projection layer
    first_projection_layer = tf.keras.layers.Dense(
        effective_embedding_size,
        activation='tanh',
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
        name="projection_layer"
    )
    dropout = tf.keras.layers.Dropout(0.2, name="projection_dropout")
    projected_1 = dropout(first_projection_layer(sentence_vector_1))
    projected_2 = dropout(first_projection_layer(sentence_vector_2))

    # Normalize the projected vectors (L2 normalization)
    normalized_1 = tf.keras.layers.Lambda(
        lambda x: tf.linalg.l2_normalize(x, axis=1), name="normalize_1"
    )(projected_1)
    normalized_2 = tf.keras.layers.Lambda(
        lambda x: tf.linalg.l2_normalize(x, axis=1), name="normalize_2"
    )(projected_2)

    # Compute Cosine Similarity
    similarity_score = tf.keras.layers.Lambda(
        lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="cosine_similarity"
    )([normalized_1, normalized_2])

    # Scale similarity from [-1, 1] to [0, 1]
    output_layer = tf.keras.layers.Lambda(
        lambda x: 0.5 * (1.0 + x), name="output_scaling"
    )(similarity_score)

    # Define the Keras Model
    model = tf.keras.Model(
        inputs=[input_1, input_2],
        outputs=output_layer,
        name="sequence_similarity_attention_model"
    )

    # Compile the model
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['mae'],
    )

    return model

**Preparació de Dades per al Model de Seqüències**

Per alimentar el model de seqüències, necessitem transformar les frases de text en un format numèric adequat. Això implica diversos passos:

1.  **`create_vocabulary_mapping` Funció**:
    *   **Propòsit**: Crea un vocabulari a partir de totes les frases del dataset i assigna un índex enter únic a cada paraula.
    *   **Funcionament**:
        1.  Compta la freqüència de cada paraula en el corpus (després de preprocessar les frases).
        2.  Ordena les paraules per freqüència de manera descendent.
        3.  Crea dos diccionaris:
            *   `word_to_idx`: Mapa cada paraula al seu índex.
            *   `idx_to_word`: Mapa cada índex a la seva paraula.
        4.  Reserva índexs especials: `0` per a `<PAD>` (padding) i `1` per a `<UNK>` (paraula desconeguda).
        5.  Limita la mida del vocabulari a `max_vocab_size`. Les paraules menys freqüents es tractaran com `<UNK>`.

2.  **`sentence_to_sequence` Funció**:
    *   **Propòsit**: Converteix una frase de text en una seqüència d'índexs de paraules, assegurant que totes les seqüències tinguin una longitud fixa.
    *   **Funcionament**:
        1.  Preprocessa la frase.
        2.  Per a cada paraula, busca el seu índex en `word_to_idx`. Si la paraula no hi és, utilitza l'índex de `<UNK>`.
        3.  **Padding/Truncament**:
            *   Si la seqüència resultant és més llarga que `max_length`, es trunca.
            *   Si és més curta, s'afegeixen índexs de `<PAD>` (valor 0) al final fins a assolir `max_length`.

3.  **`create_pretrained_embedding_matrix` Funció**:
    *   **Propòsit**: Crea una matriu de pesos per a la capa d'embedding del model de seqüències, inicialitzant-la amb vectors d'embeddings pre-entrenats (com Word2Vec).
    *   **Funcionament**:
        1.  Crea una matriu de zeros amb dimensions `(vocab_size, embedding_dim)`.
        2.  Per a cada paraula i el seu índex en `word_to_idx`:
            *   Si la paraula existeix en el diccionari d'embeddings pre-entrenats (`embeddings_dict`), copia el seu vector a la fila corresponent de la matriu d'embedding.
            *   Les paraules no trobades (incloent `<PAD>` i `<UNK>` si no tenen embeddings predefinits) mantindran vectors de zeros o els que s'hagin assignat aleatòriament si la capa d'embedding és entrenable.

Aquestes funcions permeten transformar les dades textuals en tensors numèrics que el model de Keras pot processar, alhora que permeten incorporar coneixement previ mitjançant els embeddings pre-entrenats.

In [ ]:
from typing import List, Dict, Tuple
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

# Preparació de dades per al model de seqüències
def create_vocabulary_mapping(sentences: List[str], max_vocab_size: int = 10000) -> Tuple[Dict[str, int], Dict[int, str]]:
    """
    Crea un mapatge de vocabulari paraula->índex i índex->paraula
    """
    word_counts = {}
    for sentence in sentences:
        words = preprocess_sentence(sentence)
        for word in words:
            word_counts[word] = word_counts.get(word, 0) + 1
    
    # Ordenar per freqüència i prendre les més comunes
    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Reservar índexs especials: 0=PAD, 1=UNK
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    idx_to_word = {0: "<PAD>", 1: "<UNK>"}
    
    for word, count in sorted_words[:max_vocab_size-2]:  # -2 per PAD i UNK
        idx = len(word_to_idx)
        word_to_idx[word] = idx
        idx_to_word[idx] = word
    
    return word_to_idx, idx_to_word

def sentence_to_sequence(sentence: str, word_to_idx: Dict[str, int], 
                        max_length: int = 32) -> np.ndarray:
    """
    Converteix una frase a seqüència d'índexs
    """
    words = preprocess_sentence(sentence)
    sequence = []
    
    for word in words:
        if word in word_to_idx:
            sequence.append(word_to_idx[word])
        else:
            sequence.append(word_to_idx["<UNK>"])
    
    # Padding o truncament
    if len(sequence) > max_length:
        sequence = sequence[:max_length]
    else:
        sequence.extend([word_to_idx["<PAD>"]] * (max_length - len(sequence)))
    
    return np.array(sequence)

def create_pretrained_embedding_matrix(word_to_idx: Dict[str, int], 
                                     embeddings_dict: Dict[str, np.ndarray],
                                     embedding_dim: int) -> np.ndarray:
    """
    Crea una matriu d'embeddings pre-entrenats
    """
    vocab_size = len(word_to_idx)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    for word, idx in word_to_idx.items():
        if word in embeddings_dict:
            embedding_matrix[idx] = embeddings_dict[word]
        # Les paraules no trobades mantenen vectors zero
    
    return embedding_matrix

**Entrenament i Avaluació dels Models de Seqüència**

Un cop definides les funcions de preparació de dades i l'arquitectura del model de seqüències, procedim a entrenar-lo i avaluar-lo.

**Passos del Procés:**

1.  **Creació del Vocabulari Global**:
    *   S'utilitza `create_vocabulary_mapping` amb totes les frases del dataset (`all_sentences`) per construir `word_to_idx` i `idx_to_word`. Es defineix una mida màxima de vocabulari (`max_vocab_size`) i una longitud de seqüència (`sequence_length`).

2.  **Preparació de Dades de Seqüència**:
    *   La funció `prepare_sequence_data` converteix els DataFrames d'entrenament i validació en seqüències d'índexs (`X1_train_seq`, `X2_train_seq`, `X1_val_seq`, `X2_val_seq`) i les etiquetes corresponents (`Y_train_seq`, `Y_val_seq`).
    *   **Important**: Les etiquetes `Y_train_seq` i `Y_val_seq` s'utilitzen directament. Com que el model de seqüència produeix una sortida en el rang `[0, 1]` (degut a `0.5 * (1 + cosine_similarity)`), les etiquetes originals del dataset STS-ca (rang `[0, 5]`) s'haurien d'escalar a `[0, 1]` abans de l'entrenament per a una comparació directa amb la sortida del model i un càlcul correcte de la pèrdua. Si no es fa, la pèrdua `mean_squared_error` es calcularà entre valors en escales diferents. *El codi actual no mostra aquest escalat de les etiquetes Y, la qual cosa podria afectar la interpretació de la pèrdua durant l'entrenament, tot i que la correlació de Pearson (calculada posteriorment amb les prediccions escalades) seguiria sent una mètrica vàlida per comparar el rànquing.* Per a una pràctica òptima, `Y_train_seq` i `Y_val_seq` s'haurien de dividir per 5.0.

3.  **Iteració per Dimensions d'Embedding**:
    *   S'entrenen models per a cada dimensió d'embedding disponible (50, 100, 150, 300).

4.  **Per a cada Dimensió, Tres Configuracions d'Embedding**:
    *   **Matriu d'Embeddings Pre-entrenats**: Es crea utilitzant `create_pretrained_embedding_matrix` amb els embeddings Word2Vec truncats corresponents a la dimensió actual.
    *   **Model 1: Embeddings Pre-entrenats Congelats (`trainable_embeddings=False`)**:
        *   El model utilitza els embeddings Word2Vec com a característiques fixes. La capa d'embedding no s'actualitza durant l'entrenament.
    *   **Model 2: Embeddings Pre-entrenats Entrenables (`trainable_embeddings=True`)**:
        *   El model comença amb els embeddings Word2Vec, però aquests s'ajusten (fine-tuning) durant l'entrenament per adaptar-se millor a la tasca específica de STS.
    *   **Model 3: Embeddings Aleatoris (`pretrained_weights=None`, `trainable_embeddings=True`)**:
        *   La capa d'embedding s'inicialitza amb pesos aleatoris i s'aprèn des de zero durant l'entrenament. Això serveix per avaluar la contribució dels embeddings pre-entrenats.

5.  **Entrenament i Avaluació per a cada Configuració**:
    *   Es construeix el model amb `build_model_sequence`.
    *   S'entrena el model (`model.fit`) utilitzant les dades de seqüència. No s'utilitzen callbacks d'`EarlyStopping` o `ReduceLROnPlateau` en aquest bucle, la qual cosa podria ser una millora a considerar.
    *   Es fan prediccions en el conjunt de validació. Les prediccions del model estan en el rang `[0, 1]`.
    *   **Escalat de Prediccions**: Les prediccions `Y_pred_...` (en rang `[0,1]`) es multipliquen per 5.0 per portar-les a l'escala original de les etiquetes `[0,5]` abans de calcular Pearson, MSE i MAE. Això és correcte per a l'avaluació.
    *   Es calculen les mètriques (Pearson, MSE, MAE) i es guarden els resultats.

Aquest experimentació exhaustiva permetrà comparar l'efecte de la dimensionalitat, l'ús d'embeddings pre-entrenats, i la possibilitat de fer fine-tuning d'aquests embeddings en el context d'un model basat en seqüències amb atenció.

In [ ]:
print("Preparant dades per al model de seqüències...")

# Crear vocabulari
word_to_idx, idx_to_word = create_vocabulary_mapping(all_sentences, max_vocab_size=10000)
vocab_size = len(word_to_idx)
sequence_length = 32

print(f"Vocabulari creat: {vocab_size} paraules")
print(f"Longitud de seqüència: {sequence_length}")

# Convertir dades per a models de seqüència
def prepare_sequence_data(df):
    X1_seq, X2_seq, Y_seq = [], [], []
    for _, row in df.iterrows():
        seq1 = sentence_to_sequence(row['sentence_1'], word_to_idx, sequence_length)
        seq2 = sentence_to_sequence(row['sentence_2'], word_to_idx, sequence_length)
        X1_seq.append(seq1)
        X2_seq.append(seq2)
        # IMPORTANT: Les etiquetes Y_seq haurien d'estar escalades a [0,1] si el model prediu en aquest rang
        # per a un càlcul correcte de la pèrdua durant l'entrenament.
        # Per exemple: Y_seq.append(row['label'] / 5.0)
        # Aquí es mantenen en [0,5] i les prediccions s'escalen posteriorment per a mètriques.
        Y_seq.append(row['label']) 
    return np.array(X1_seq), np.array(X2_seq), np.array(Y_seq)

X1_train_seq, X2_train_seq, Y_train_seq = prepare_sequence_data(train_df)
X1_val_seq, X2_val_seq, Y_val_seq = prepare_sequence_data(val_df)

# Escalar Y_val_seq a [0,1] per a una comparació més directa amb les prediccions del model abans d'escalar-les a [0,5]
# Y_val_seq_scaled = Y_val_seq / 5.0 # Descomentar si es vol utilitzar per a mètriques directes amb sortida [0,1]

print(f"Dades de seqüència preparades: {X1_train_seq.shape}")

# Entrenar models de seqüència per diferents dimensions
sequence_results = {}

for embedding_dim in [50, 100, 150, 300]:
    if embedding_dim in truncated_embeddings:
        print(f"\n=== MODELS DE SEQÜÈNCIA {embedding_dim}D ===")
        
        # Preparar matriu d'embeddings pre-entrenats
        pretrained_matrix = create_pretrained_embedding_matrix(
            word_to_idx, truncated_embeddings[embedding_dim], embedding_dim
        )
        
        # Model 1: Embeddings pre-entrenats (frozen)
        print(f"Entrenant model amb embeddings pre-entrenats frozen ({embedding_dim}D)...")
        model_seq_frozen = build_model_sequence(
            vocab_size=vocab_size,
            embedding_dim=embedding_dim,
            sequence_length=sequence_length,
            pretrained_weights=pretrained_matrix,
            trainable_embeddings=False
        )
        
        history_frozen = model_seq_frozen.fit(
            [X1_train_seq, X2_train_seq], Y_train_seq / 5.0, # Escalar Y_train_seq a [0,1]
            validation_data=([X1_val_seq, X2_val_seq], Y_val_seq / 5.0), # Escalar Y_val_seq a [0,1]
            epochs=30,
            batch_size=32,
            verbose=0
        )
        
        Y_pred_frozen_raw = model_seq_frozen.predict([X1_val_seq, X2_val_seq]).flatten() # Surtida en [0,1]
        Y_pred_frozen = Y_pred_frozen_raw * 5.0 # Escalar a [0,5] per a mètriques amb Y_val_seq original
        
        pearson_frozen, _ = pearsonr(Y_val_seq, Y_pred_frozen) # Comparar amb Y_val_seq original [0,5]
        mse_frozen = mean_squared_error(Y_val_seq, Y_pred_frozen)
        mae_frozen = mean_absolute_error(Y_val_seq, Y_pred_frozen)
        
        print(f"  Frozen - Pearson: {pearson_frozen:.3f}, MSE: {mse_frozen:.3f}, MAE: {mae_frozen:.3f}")
        
        # Model 2: Embeddings pre-entrenats (trainable)
        print(f"Entrenant model amb embeddings pre-entrenats trainable ({embedding_dim}D)...")
        model_seq_trainable = build_model_sequence(
            vocab_size=vocab_size,
            embedding_dim=embedding_dim,
            sequence_length=sequence_length,
            pretrained_weights=pretrained_matrix,
            trainable_embeddings=True
        )
        
        history_trainable = model_seq_trainable.fit(
            [X1_train_seq, X2_train_seq], Y_train_seq / 5.0, # Escalar Y_train_seq a [0,1]
            validation_data=([X1_val_seq, X2_val_seq], Y_val_seq / 5.0), # Escalar Y_val_seq a [0,1]
            epochs=30,
            batch_size=32,
            verbose=0
        )
        
        Y_pred_trainable_raw = model_seq_trainable.predict([X1_val_seq, X2_val_seq]).flatten() # Surtida en [0,1]
        Y_pred_trainable = Y_pred_trainable_raw * 5.0 # Escalar a [0,5]
        
        pearson_trainable, _ = pearsonr(Y_val_seq, Y_pred_trainable)
        mse_trainable = mean_squared_error(Y_val_seq, Y_pred_trainable)
        mae_trainable = mean_absolute_error(Y_val_seq, Y_pred_trainable)
        
        print(f"  Trainable - Pearson: {pearson_trainable:.3f}, MSE: {mse_trainable:.3f}, MAE: {mae_trainable:.3f}")
        
        # Model 3: Embeddings aleatoris
        
        print(f"Entrenant model amb embeddings aleatoris ({embedding_dim}D)...")
        model_seq_random = build_model_sequence(
            vocab_size=vocab_size,
            embedding_dim=embedding_dim,
            sequence_length=sequence_length,
            pretrained_weights=None,
            trainable_embeddings=True
        )
        
        history_random = model_seq_random.fit(
            [X1_train_seq, X2_train_seq], Y_train_seq / 5.0, # Escalar Y_train_seq a [0,1]
            validation_data=([X1_val_seq, X2_val_seq], Y_val_seq / 5.0), # Escalar Y_val_seq a [0,1]
            epochs=30,
            batch_size=32,
            verbose=0
        )
        
        Y_pred_random_raw = model_seq_random.predict([X1_val_seq, X2_val_seq]).flatten() # Surtida en [0,1]
        Y_pred_random = Y_pred_random_raw * 5.0 # Escalar a [0,5]

        pearson_random, _ = pearsonr(Y_val_seq, Y_pred_random)
        mse_random = mean_squared_error(Y_val_seq, Y_pred_random)
        mae_random = mean_absolute_error(Y_val_seq, Y_pred_random)
        
        print(f"  Random - Pearson: {pearson_random:.3f}, MSE: {mse_random:.3f}, MAE: {mae_random:.3f}")
        
        sequence_results[embedding_dim] = {
            'frozen': {'model': model_seq_frozen, 'pearson': pearson_frozen, 'mse': mse_frozen, 'mae': mae_frozen},
            'trainable': {'model': model_seq_trainable, 'pearson': pearson_trainable, 'mse': mse_trainable, 'mae': mae_trainable},
            'random': {'model': model_seq_random, 'pearson': pearson_random, 'mse': mse_random, 'mae': mae_random}
        }
    else: # Aquest 'else' sembla que podria ser un error lògic si truncated_embeddings no conté la clau
          # ja que model_seq_frozen, etc. no estarien definits. S'hauria de gestionar millor.
          # Per ara, assumim que si embedding_dim està en la llista [50,100,150,300], estarà a truncated_embeddings.
        pass # No hi hauria d'haver un 'else' aquí si el 'if embedding_dim in truncated_embeddings' gestiona tots els casos.
             # Si es volgués gestionar el cas on no hi ha embeddings per a una dimensió, s'hauria de fer explícitament.
             # El codi original tenia un error aquí que he corregit parcialment comentant l'else.

**Anàlisi dels Resultats dels Models de Seqüència**

La taula següent presenta una comparació del rendiment (correlació de Pearson) dels models de seqüència sota diferents configuracions d'embeddings i dimensions.

**Observacions Clau:**
*   **Frozen vs. Trainable vs. Random**:
    *   **Frozen**: Utilitzar embeddings pre-entrenats sense modificar-los sol donar un bon punt de partida, aprofitant el coneixement general capturat per Word2Vec.
    *   **Trainable**: Permetre que els embeddings pre-entrenats s'ajustin (fine-tuning) durant l'entrenament pot millorar el rendiment, ja que s'adapten més a la tasca específica de STS i al dataset. No obstant això, si el dataset d'entrenament és petit, hi ha risc de sobreajustament.
    *   **Random**: Entrenar embeddings des de zero sol ser el més difícil i requereix més dades. S'espera que aquesta configuració tingui el rendiment més baix, especialment amb datasets de mida moderada, ja que no aprofita cap coneixement previ.
*   **Impacte de la Dimensió**: Similar als models anteriors, la dimensionalitat dels embeddings pot influir en el rendiment. Cal observar si la tendència es manté amb aquesta arquitectura més complexa.
*   **Comparació General**: Aquests resultats es compararan amb els del Model 1 (agregat) i els baselines per avaluar si l'ús de seqüències i atenció aporta millores significatives.

Les mètriques detallades (Pearson, MSE, MAE) per a cada configuració s'afegeixen al DataFrame `df_results` per a una anàlisi global posterior.

*Nota sobre l'escalat de les etiquetes*: En la cel·la anterior, s'ha modificat el codi per escalar les etiquetes `Y_train_seq` i `Y_val_seq` a l'interval `[0,1]` quan s'alimenten a la funció `fit` del model (`Y_train_seq / 5.0`). Això és perquè la sortida del model `build_model_sequence` està en l'interval `[0,1]`. Les prediccions (`Y_pred_..._raw`) també estan en `[0,1]` i després s'escalen a `[0,5]` (`Y_pred_...`) per calcular les mètriques amb les etiquetes de validació originals (`Y_val_seq`) que estan en `[0,5]`. Aquest ajust és important per a la correcta interpretació de la funció de pèrdua durant l'entrenament.*

In [ ]:
# Comparació dels models de seqüència
print("\n=== COMPARACIÓ MODELS DE SEQÜÈNCIA ===")

sequence_comparison_data = []
sequence_metrics = []

for dim in [50, 100, 150, 300]:
    if dim in sequence_results:
        seq = sequence_results[dim]
        # Comparació Pearson
        frozen_r = seq['frozen']['pearson']
        trainable_r = seq['trainable']['pearson']
        row = {
            'Dimensions': f'{dim}D',
            'Frozen': frozen_r,
            'Trainable': trainable_r,
        }
        if 'random' in seq: # Comprovar si 'random' existeix abans d'accedir-hi
            if seq['random'] is not None and 'pearson' in seq['random']: # Doble verificació
                 row['Random'] = seq['random']['pearson']
            else:
                row['Random'] = np.nan # O un altre valor per indicar que no hi ha dades
        else:
            row['Random'] = np.nan

        sequence_comparison_data.append(row)
        
        # Mètriques detallades
        for key in ['frozen', 'trainable', 'random']:
            if key in seq and seq[key] is not None: # Comprovar si la clau i el seu valor existeixen
                sequence_metrics.append({
                    'Model': f"Model Seqüència ({key.capitalize()})",
                    'Dimensions': f'{dim}D',
                    'Pearson': seq[key]['pearson'],
                    'MSE': seq[key]['mse'],
                    'MAE': seq[key]['mae']
                })

# Mostrar taules
display(pd.DataFrame(sequence_comparison_data).style.hide(axis="index"))
df_results = pd.concat([df_results, pd.DataFrame(sequence_metrics)], ignore_index=True)

## 8. Anàlisi Comparativa i Visualització de Resultats (STS)

Aquesta secció té com a objectiu consolidar i visualitzar els resultats obtinguts fins ara amb els models basats en Word2Vec (baselines, model agregat, model de seqüència).

**Resum de Resultats:**
Primer, es presenta un resum textual dels resultats de correlació de Pearson per a:
*   **Baselines Cosinus**: Comparant la mitjana simple amb la mitjana ponderada per TF-IDF per a cada dimensió.
*   **Models de Regressió Agregats**: Mostrant Pearson, MSE i MAE per a cada dimensió.
*   **Models de Seqüència**: Mostrant Pearson, MSE i MAE per a cada dimensió i cada configuració d'embedding (Frozen, Trainable, Random).

A continuació, es mostra la taula completa `df_results`, que acumula totes aquestes mètriques, permetent una comparació directa entre tots els models i configuracions provats fins ara.

**Visualitzacions:**
Es generen diversos gràfics per facilitar la interpretació dels resultats:

1.  **Comparació de Correlació de Pearson per Dimensions**:
    *   Un diagrama de barres que compara la correlació de Pearson dels models Baseline (Simple i TF-IDF) i el Model Agregat a través de les diferents dimensions d'embedding (50D, 100D, 150D, 300D).
    *   **Anàlisi Esperada**: Aquest gràfic ajudarà a visualitzar quin tipus de model (baseline simple, baseline TF-IDF, agregat) funciona millor en general i com la dimensionalitat afecta cada un.

2.  **Comparació de MSE per Dimensions**:
    *   Similar a l'anterior, però mostrant l'Error Quadràtic Mig (MSE). Valors més baixos són millors.
    *   **Anàlisi Esperada**: Complementa el gràfic de Pearson, mostrant una altra perspectiva del rendiment del model.

3.  **Prediccions vs. Valors Reals (Millor Model Agregat)**:
    *   Un diagrama de dispersió que enfronta les prediccions del millor model agregat (seleccionat per la correlació de Pearson més alta) contra els valors reals de similitud del conjunt de validació. Una línia diagonal (y=x) indica una predicció perfecta.
    *   **Anàlisi Esperada**: Permet avaluar visualment com de bé s'ajusten les prediccions als valors reals i si hi ha patrons sistemàtics en els errors.

4.  **Distribució d'Errors (Millor Model Agregat)**:
    *   Un histograma que mostra la distribució dels errors de predicció (Predicció - Real) per al millor model agregat.
    *   **Anàlisi Esperada**: Idealment, la distribució hauria d'estar centrada en zero i ser el més estreta possible. Això indica si el model tendeix a sobreestimar o subestimar i la magnitud típica dels errors.

Aquestes anàlisis i visualitzacions són crucials per entendre les fortaleses i debilitats dels models desenvolupats i per guiar les decisions en experiments futurs.

In [ ]:
# Resum simplificat de resultats
if kv_model is not None: # Assegurar que els resultats existeixen
    print("=== RESUM DE RESULTATS (Word2Vec based) ===\n")

    print("BASELINES COSINUS:")
    for dim_str in ['50D', '100D', '150D', '300D']:
        simple_df = df_results[(df_results['Model'] == 'Baseline Cosinus Simple') & (df_results['Dimensions'] == dim_str)]
        tfidf_df = df_results[(df_results['Model'] == 'Baseline Cosinus TF-IDF') & (df_results['Dimensions'] == dim_str)]
        
        pearson_simple = simple_df['Pearson'].values[0] if not simple_df.empty else 'N/A'
        pearson_tfidf = tfidf_df['Pearson'].values[0] if not tfidf_df.empty else 'N/A'
        
        if pearson_simple != 'N/A' or pearson_tfidf != 'N/A':
             print(f"  {dim_str} - Simple: {pearson_simple if isinstance(pearson_simple, str) else f'{pearson_simple:.3f}'}, TF-IDF: {pearson_tfidf if isinstance(pearson_tfidf, str) else f'{pearson_tfidf:.3f}'}")

    print("\nMODELS DE REGRESSIÓ AGREGATS:")
    for dim_str in ['50D', '100D', '150D', '300D']:
        aggr_df = df_results[(df_results['Model'] == 'Model Agregat') & (df_results['Dimensions'] == dim_str)]
        if not aggr_df.empty:
            row = aggr_df.iloc[0]
            print(f"  {dim_str} - Pearson: {row['Pearson']:.3f}, MSE: {row['MSE']:.3f}, MAE: {row['MAE']:.3f}")

    print("\nMODELS DE SEQÜÈNCIA:")
    for dim_str in ['50D', '100D', '150D', '300D']:
        print(f"  --- Dimensions: {dim_str} ---")
        for mode in ['Frozen', 'Trainable', 'Random']:
            seq_df = df_results[(df_results['Model'] == f'Model Seqüència ({mode})') & (df_results['Dimensions'] == dim_str)]
            if not seq_df.empty:
                row = seq_df.iloc[0]
                print(f"    {mode}: Pearson: {row['Pearson']:.3f}, MSE: {row['MSE']:.3f}, MAE: {row['MAE']:.3f}")
            # else: # Opcional: indicar si no hi ha dades per a alguna combinació
            #     print(f"    {mode}: N/A")


    print("\n=== TAULA DE RESULTATS (Word2Vec based) ===")
    # Filtrar per mostrar només els resultats dels models basats en Word2Vec fins ara
    word2vec_models_patterns = ['Baseline Cosinus', 'Model Agregat', 'Model Seqüència']
    df_results_word2vec = df_results[df_results['Model'].str.contains('|'.join(word2vec_models_patterns), na=False)]
    display(df_results_word2vec.sort_values(by='Pearson', ascending=False))
else:
    print("kv_model no està carregat, no es poden mostrar resultats detallats de Word2Vec.")
    display(df_results.sort_values(by='Pearson', ascending=False))


In [ ]:
# Visualitzacions
if kv_model is not None and baseline_results and aggregated_results: # Assegurar que les dades per als gràfics existeixen
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle("Anàlisi de Models Basats en Word2Vec (Mitjana/Agregats)", fontsize=16)
    
    # Gràfic 1: Comparació de Pearson per dimensions
    baseline_dims = sorted([d for d in [50, 100, 150, 300] if d in baseline_results and d in aggregated_results])
    
    if baseline_dims: # Només crear gràfics si hi ha dimensions comunes
        baseline_simple_pearson = [baseline_results[d]['simple']['pearson'] for d in baseline_dims]
        baseline_tfidf_pearson = [baseline_results[d]['tfidf']['pearson'] for d in baseline_dims]
        aggregated_pearson_vals = [aggregated_results[d]['pearson'] for d in baseline_dims]
        
        x = np.arange(len(baseline_dims))
        width = 0.25
        
        axes[0,0].bar(x - width, baseline_simple_pearson, width, label='Baseline Simple', alpha=0.8)
        axes[0,0].bar(x, baseline_tfidf_pearson, width, label='Baseline TF-IDF', alpha=0.8)
        axes[0,0].bar(x + width, aggregated_pearson_vals, width, label='Model Agregat', alpha=0.8)
        
        axes[0,0].set_xlabel('Dimensions dels Embeddings Word2Vec')
        axes[0,0].set_ylabel('Correlació de Pearson')
        axes[0,0].set_title('Correlació de Pearson per Dimensions')
        axes[0,0].set_xticks(x)
        axes[0,0].set_xticklabels([f'{d}D' for d in baseline_dims])
        axes[0,0].legend()
        axes[0,0].grid(True, linestyle='--', alpha=0.7)
        
        # Gràfic 2: MSE per dimensions
        baseline_mse_simple = [baseline_results[d]['simple']['mse'] for d in baseline_dims]
        baseline_mse_tfidf = [baseline_results[d]['tfidf']['mse'] for d in baseline_dims]
        aggregated_mse_vals = [aggregated_results[d]['mse'] for d in baseline_dims]
        
        axes[0,1].bar(x - width, baseline_mse_simple, width, label='Baseline Simple', alpha=0.8)
        axes[0,1].bar(x, baseline_mse_tfidf, width, label='Baseline TF-IDF', alpha=0.8)
        axes[0,1].bar(x + width, aggregated_mse_vals, width, label='Model Agregat', alpha=0.8)
        
        axes[0,1].set_xlabel('Dimensions dels Embeddings Word2Vec')
        axes[0,1].set_ylabel('Mean Squared Error (MSE)')
        axes[0,1].set_title('MSE per Dimensions')
        axes[0,1].set_xticks(x)
        axes[0,1].set_xticklabels([f'{d}D' for d in baseline_dims])
        axes[0,1].legend()
        axes[0,1].grid(True, linestyle='--', alpha=0.7)
        
        # Gràfic 3: Prediccions vs Valors Reals (millor model agregat)
        # Trobar el millor model agregat basat en Pearson
        if aggregated_results: # Assegurar que aggregated_results no està buit
            best_model_dim_aggr = max(aggregated_results.keys(), key=lambda k: aggregated_results[k]['pearson'] if k in aggregated_results else -1)
            if best_model_dim_aggr != -1 and best_model_dim_aggr in aggregated_results:
                best_predictions_aggr = aggregated_results[best_model_dim_aggr]['predictions']
                real_values_val = val_df['label'].values[:len(best_predictions_aggr)] # Assegurar mateixa longitud
                
                axes[1,0].scatter(real_values_val, best_predictions_aggr, alpha=0.6, edgecolors='w', linewidth=0.5)
                axes[1,0].plot([0, 5], [0, 5], 'r--', alpha=0.8, label="Predicció Perfecta") # Línia y=x
                axes[1,0].set_xlabel('Valors Reals (Similitud)')
                axes[1,0].set_ylabel('Prediccions del Model')
                axes[1,0].set_title(f'Millor Model Agregat ({best_model_dim_aggr}D) - Prediccions vs Reals')
                axes[1,0].legend()
                axes[1,0].grid(True, linestyle='--', alpha=0.7)
                axes[1,0].set_xlim(0, 5)
                axes[1,0].set_ylim(0, 5)

                # Gràfic 4: Distribució d'errors
                errors_aggr = best_predictions_aggr - real_values_val
                sns.histplot(errors_aggr, bins=30, ax=axes[1,1], kde=True)
                axes[1,1].axvline(x=0, color='red', linestyle='--', alpha=0.8, label="Error Zero")
                axes[1,1].set_xlabel('Error (Predicció - Real)')
                axes[1,1].set_ylabel('Freqüència')
                axes[1,1].set_title(f'Distribució d\'Errors (Model Agregat {best_model_dim_aggr}D)')
                axes[1,1].legend()
                axes[1,1].grid(True, linestyle='--', alpha=0.7)
            else:
                axes[1,0].text(0.5, 0.5, "No hi ha dades per al gràfic de prediccions", ha='center', va='center')
                axes[1,1].text(0.5, 0.5, "No hi ha dades per al gràfic d'errors", ha='center', va='center')

        else:
            axes[1,0].text(0.5, 0.5, "No hi ha resultats agregats per mostrar", ha='center', va='center')
            axes[1,1].text(0.5, 0.5, "No hi ha resultats agregats per mostrar", ha='center', va='center')

    else:
        for i in range(2):
            for j in range(2):
                axes[i,j].text(0.5, 0.5, "No hi ha dades suficients per generar els gràfics.", ha='center', va='center')

    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Ajustar per al títol general
    plt.show()
else:
    print("No es poden generar les visualitzacions ja que falten resultats (kv_model, baseline_results o aggregated_results no disponibles).")

## 9. Experimentació Avançada (Opcional)

En aquesta secció, explorem tècniques addicionals i alternatives per a la tasca de STS, anant més enllà dels models basats exclusivament en Word2Vec i arquitectures simples.

**Experiments Realitzats:**

1.  **Baseline One-Hot Encoding**:
    *   **Propòsit**: Implementar un baseline molt simple basat en representacions One-Hot (o més aviat, Bag-of-Words binaritzat) de les frases.
    *   **Mètode**: S'utilitza `CountVectorizer` per convertir les frases en vectors binaris on cada posició representa la presència o absència d'una paraula d'un vocabulari limitat. La similitud es calcula amb la distància cosinus. S'aplica un escalat millorat per ajustar les prediccions al rang de les etiquetes reals.
    *   **Anàlisi Esperada**: Aquest model sol tenir un rendiment inferior als basats en embeddings densos, però serveix com a referència mínima.

2.  **Experimentació Sistemàtica amb Hiperparàmetres (Model Agregat)**:
    *   **Propòsit**: Investigar com diferents hiperparàmetres afecten el rendiment del Model 1 (Regressió amb Embeddings Agregats).
    *   **Mètode**: Es proven diverses combinacions de `hidden_size` (mida de les capes ocultes), `dropout_rate` (taxa de dropout) i `learning_rate` (taxa d'aprenentatge de l'optimitzador Adam). S'utilitzen embeddings de 150D per a aquest experiment per equilibrar rendiment i velocitat.
    *   **Anàlisi Esperada**: Identificar les millors configuracions d'hiperparàmetres i entendre la sensibilitat del model a cadascun d'ells. Es visualitza l'impacte individual de cada hiperparàmetre en la correlació de Pearson.

3.  **Ús d'Embeddings de spaCy**:
    *   **Propòsit**: Avaluar el rendiment utilitzant embeddings de frases proporcionats per la llibreria spaCy, específicament del model `ca_core_news_md`. Aquests embeddings són contextuals a nivell de document/frase.
    *   **Mètode**: Per a cada frase, s'obté el seu vector (`doc.vector`) de spaCy. Aquests vectors es trunquen a les mateixes dimensions que els Word2Vec (50D, 100D, 150D, 300D) per a una comparació més directa, i es calcula la similitud cosinus.
    *   **Anàlisi Esperada**: Comparar el rendiment dels embeddings de spaCy amb els de Word2Vec. Els embeddings de spaCy, tot i ser de propòsit general, podrien capturar millor el context.

4.  **Ús d'Embeddings de RoBERTa (Model Base)**:
    *   **Propòsit**: Utilitzar un model Transformer pre-entrenat (RoBERTa, `projecte-aina/roberta-base-ca-v2`) per extreure embeddings de frases.
    *   **Mètode**: Les frases es passen a través del model RoBERTa, i s'utilitza l'embedding del token especial `[CLS]` (o la mitjana dels embeddings dels tokens de la última capa oculta) com a representació de la frase. Aquests embeddings es trunquen a diferents dimensions i es calcula la similitud cosinus. Es processen les frases en lots per eficiència.
    *   **Anàlisi Esperada**: Els models Transformer solen oferir representacions semàntiques molt riques. S'espera un bon rendiment, possiblement superant els mètodes anteriors.

5.  **Ús d'un Model RoBERTa Fine-Tuned per STS**:
    *   **Propòsit**: Avaluar un model RoBERTa (`projecte-aina/roberta-base-ca-v2-cased-sts`) que ha estat específicament fine-tuned (ajustat) per a la tasca de Semantic Textual Similarity en català.
    *   **Mètode**: S'utilitza la `pipeline` de `transformers` per a `text-classification` (que en aquest cas s'adapta a STS). El model directament prediu una puntuació de similitud per a un parell de frases.
    *   **Anàlisi Esperada**: Aquest model hauria de ser el que ofereixi el millor rendiment, ja que ha estat entrenat específicament per a aquesta tasca i domini lingüístic.

Aquests experiments proporcionaran una visió més àmplia de les capacitats de diferents tècniques d'embedding i modelatge per a STS en català.

In [ ]:
# Baseline One-Hot (vocabulari limitat)
from sklearn.feature_extraction.text import CountVectorizer

def evaluate_onehot(df: pd.DataFrame, max_features: int = 1000) -> Dict[str, float]:
    all_sents = df['sentence_1'].tolist() + df['sentence_2'].tolist()
    # Utilitzar preprocess_sentence per coherència, tot i que CountVectorizer ja fa lowercase.
    processed_sents = [' '.join(preprocess_sentence(s)) for s in all_sents]
    
    vectorizer = CountVectorizer(max_features=max_features, binary=True) # lowercase ja ho fa preprocess_sentence
    vectorizer.fit(processed_sents)
    
    similarities_raw = []
    true_scores = df['label'].values

    for _, row in df.iterrows():
        # Preprocessar frases abans de transformar
        s1_processed = ' '.join(preprocess_sentence(row['sentence_1']))
        s2_processed = ' '.join(preprocess_sentence(row['sentence_2']))

        vec1 = vectorizer.transform([s1_processed]).toarray()[0]
        vec2 = vectorizer.transform([s2_processed]).toarray()[0]
        
        if np.sum(vec1) == 0 or np.sum(vec2) == 0: # Si alguna frase no té paraules del vocabulari
            sim = 0.0
        else:
            # Similitud cosinus: 1 - distance.cosine
            # distance.cosine = dot(u,v) / (||u|| * ||v||) -- No, cosine distance és 1 - cos_sim
            # cos_sim = dot(u,v) / (||u|| * ||v||)
            # Aquí, com és binari, ||u|| = sqrt(sum(u_i^2)) = sqrt(num_ones_in_u)
            # dot(u,v) = nombre de paraules comunes
            
            # sklearn.metrics.pairwise.cosine_similarity retorna la similitud, no la distància
            # from sklearn.metrics.pairwise import cosine_similarity
            # sim = cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))[0,0]
            # Però per mantenir la coherència amb l'ús de scipy.spatial.distance.cosine:
            sim = 1 - cosine(vec1, vec2) if not (np.all(vec1 == 0) or np.all(vec2 == 0)) else 0.0

        similarities_raw.append(sim)
    
    similarities_raw = np.array(similarities_raw)
    
    # Escalado mejorado: normalizar a la distribución de labels
    min_label, max_label = true_scores.min(), true_scores.max()
    
    # Evitar divisió per zero si totes les similituds són iguals
    min_sim_raw, max_sim_raw = similarities_raw.min(), similarities_raw.max()
    
    if max_sim_raw > min_sim_raw:
        # Escalar linealment les similituds [0,1] (o rang similar) al rang de les etiquetes [min_label, max_label]
        similarities_scaled = (similarities_raw - min_sim_raw) / (max_sim_raw - min_sim_raw) * (max_label - min_label) + min_label
    elif len(np.unique(similarities_raw)) == 1: # Totes les similituds són iguals
         # Si totes les similituds són iguals, assignar la mitjana de les etiquetes reals
        similarities_scaled = np.full_like(similarities_raw, np.mean(true_scores))
    else: # Cas extrem, potser totes les frases són OOV o similars
        similarities_scaled = np.full_like(similarities_raw, min_label)


    # Assegurar que les prediccions escalades estiguin dins del rang de les etiquetes
    similarities_scaled = np.clip(similarities_scaled, min_label, max_label)

    # Calcular métricas
    pearson_corr, _ = pearsonr(true_scores, similarities_scaled)
    mse = mean_squared_error(true_scores, similarities_scaled)
    mae = mean_absolute_error(true_scores, similarities_scaled)
    
    return {'pearson': pearson_corr, 'mse': mse, 'mae': mae, 'predictions': similarities_scaled}

# Evaluar con escalado mejorado
print("=== BASELINE ONE-HOT ENCODING (Bag-of-Words Binari) ===")
onehot_results_list = [] # Canviat el nom per evitar conflictes
onehot_dims_vocab = [500, 1000, 2000, 5000] # Mides de vocabulari a provar

for max_features_vocab in onehot_dims_vocab:
    print(f"Avaluant One-Hot amb max_features = {max_features_vocab}")
    res_onehot = evaluate_onehot(val_df, max_features=max_features_vocab)
    onehot_results_list.append({
        'Model': 'Baseline One-Hot (Escalat)',
        'Dimensions': f'{max_features_vocab} features', # Etiqueta més clara
        'Pearson': res_onehot['pearson'],
        'MSE': res_onehot['mse'],
        'MAE': res_onehot['mae']
    })

df_onehot_results = pd.DataFrame(onehot_results_list) # Canviat el nom
if not df_onehot_results.empty:
    display(df_onehot_results.style.hide(axis="index"))
    df_results = pd.concat([df_results, df_onehot_results], ignore_index=True)
else:
    print("No s'han generat resultats per al baseline One-Hot.")


In [ ]:
# Experimentació sistemàtica amb hiperparàmetres
print("=== EXPERIMENTACIÓ AMB HIPERPARÀMETRES (Model Agregat) ===")

hyperparams_results = []

# Provar diferents combinacions d'hiperparàmetres
hidden_sizes = [64, 128, 256]
dropout_rates = [0.2, 0.3, 0.5]
learning_rates = [0.0005, 0.001, 0.002]

# Indicador per saber si s'executa l'experiment
experiment_executed = False

if kv_model is not None and 150 in truncated_embeddings: # Assegurar que els embeddings necessaris existeixen
    print("Experimentant amb diferents hiperparàmetres per al Model Agregat (Embeddings 150D)...")
    experiment_executed = True

    # Preparar dades una sola vegada (per a 150D)
    X1_train_hyper, X2_train_hyper, Y_train_hyper = prepare_aggregated_data(
        train_df, truncated_embeddings[150], 150
    )
    X1_val_hyper, X2_val_hyper, Y_val_hyper = prepare_aggregated_data(
        val_df, truncated_embeddings[150], 150
    )

    for hidden_size in hidden_sizes:
        for dropout_rate in dropout_rates:
            for lr in learning_rates:
                print(f"Provant: hidden_size={hidden_size}, dropout={dropout_rate}, lr={lr}")
                
                # Construir model amb hiperparàmetres específics
                model_hyper = build_model_aggregated(
                    embedding_dim=150, 
                    hidden_size=hidden_size, 
                    dropout_rate=dropout_rate
                )
                
                # Recompilar amb learning rate específic
                model_hyper.compile(
                    loss='mean_squared_error',
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr), # Aplicar el learning rate actual
                    metrics=['mae', tf.keras.metrics.RootMeanSquaredError()] # Mètriques consistents
                )
                
                # Callbacks per a l'entrenament d'hiperparàmetres
                early_stopping_hyper = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=5, restore_best_weights=True, verbose=0 # Menys paciència
                )
                reduce_lr_hyper = tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=0 # Menys paciència
                )

                # Entrenament ràpid
                history_hyper = model_hyper.fit(
                    [X1_train_hyper, X2_train_hyper], Y_train_hyper,
                    validation_data=([X1_val_hyper, X2_val_hyper], Y_val_hyper),
                    epochs=25,  # Menys èpoques per rapidesa en la cerca d'hiperparàmetres
                    batch_size=32,
                    callbacks=[early_stopping_hyper, reduce_lr_hyper],
                    verbose=0 # Menys verbositat durant la cerca
                )
                
                # Avaluació
                Y_pred_hyper = model_hyper.predict([X1_val_hyper, X2_val_hyper]).flatten()
                pearson_hyper, _ = pearsonr(Y_val_hyper, Y_pred_hyper)
                mse_hyper = mean_squared_error(Y_val_hyper, Y_pred_hyper)
                mae_hyper = mean_absolute_error(Y_val_hyper, Y_pred_hyper) # Afegir MAE
                
                hyperparams_results.append({
                    'hidden_size': hidden_size,
                    'dropout_rate': dropout_rate,
                    'learning_rate': lr,
                    'pearson': pearson_hyper,
                    'mse': mse_hyper,
                    'mae': mae_hyper, # Guardar MAE
                    'final_val_loss': min(history_hyper.history['val_loss']) if 'val_loss' in history_hyper.history else np.nan,
                    'best_val_mae': min(history_hyper.history['val_mae']) if 'val_mae' in history_hyper.history else np.nan
                })
else:
    print("Saltant l'experimentació d'hiperparàmetres: kv_model no carregat o embeddings de 150D no disponibles.")


if experiment_executed and hyperparams_results: # Només mostrar resultats si l'experiment s'ha executat
    # Crear DataFrame amb resultats
    df_hyperparams = pd.DataFrame(hyperparams_results)

    print("\nMillors configuracions d'hiperparàmetres (ordenades per Pearson descendent):")
    top_configs = df_hyperparams.sort_values('pearson', ascending=False).head(10)
    display(top_configs[['hidden_size', 'dropout_rate', 'learning_rate', 'pearson', 'mse', 'mae', 'final_val_loss']])

    print("\nImpacte individual dels hiperparàmetres en la Correlació de Pearson Mitjana:")

    # Agrupar per hidden_size
    hidden_impact = df_hyperparams.groupby('hidden_size')['pearson'].agg(['mean', 'std', 'count'])
    print("\nImpacte de Hidden Size:")
    display(hidden_impact)

    # Agrupar per dropout_rate
    dropout_impact = df_hyperparams.groupby('dropout_rate')['pearson'].agg(['mean', 'std', 'count'])
    print("\nImpacte de Dropout Rate:")
    display(dropout_impact)

    # Agrupar per learning_rate
    lr_impact = df_hyperparams.groupby('learning_rate')['pearson'].agg(['mean', 'std', 'count'])
    print("\nImpacte de Learning Rate:")
    display(lr_impact)

    # Visualització dels resultats
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle("Impacte dels Hiperparàmetres en la Correlació de Pearson (Model Agregat 150D)", fontsize=16)

    # Gràfic 1: Hidden Size
    if not hidden_impact.empty:
        hidden_impact.plot(kind='bar', y='mean', yerr='std', ax=axes[0], capsize=4, legend=False, alpha=0.7)
        axes[0].set_title('Hidden Size')
        axes[0].set_ylabel('Pearson Correlation (mitjana ± std)')
        axes[0].tick_params(axis='x', rotation=0)
        axes[0].grid(True, linestyle='--', alpha=0.7)

    # Gràfic 2: Dropout Rate
    if not dropout_impact.empty:
        dropout_impact.plot(kind='bar', y='mean', yerr='std', ax=axes[1], capsize=4, legend=False, alpha=0.7)
        axes[1].set_title('Dropout Rate')
        axes[1].set_ylabel('Pearson Correlation (mitjana ± std)')
        axes[1].tick_params(axis='x', rotation=0)
        axes[1].grid(True, linestyle='--', alpha=0.7)

    # Gràfic 3: Learning Rate
    if not lr_impact.empty:
        lr_impact.plot(kind='bar', y='mean', yerr='std', ax=axes[2], capsize=4, legend=False, alpha=0.7)
        axes[2].set_title('Learning Rate')
        axes[2].set_ylabel('Pearson Correlation (mitjana ± std)')
        axes[2].tick_params(axis='x', rotation=45)
        axes[2].grid(True, linestyle='--', alpha=0.7)
        # Formatar els valors de l'eix x per a learning rate per a millor llegibilitat
        axes[2].set_xticklabels([f'{x:.4f}' for x in lr_impact.index])


    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

elif experiment_executed and not hyperparams_results:
    print("L'experimentació d'hiperparàmetres s'ha intentat executar però no ha generat resultats.")

In [ ]:
import spacy

# Carregar model de spaCy per al català
# És important tenir el model descarregat: python -m spacy download ca_core_news_md
print("\n=== AVALUACIÓ AMB EMBEDDINGS DE SPACY (ca_core_news_md) ===")
try:
    nlp = spacy.load("ca_core_news_md")
    spacy_available = True
except OSError:
    print("Model spaCy 'ca_core_news_md' no trobat. Si us plau, descarrega'l amb:")
    print("python -m spacy download ca_core_news_md")
    print("Saltant l'avaluació amb spaCy.")
    spacy_available = False

spacy_results_list = []

if spacy_available:
    # Dimensions a provar per als embeddings de spaCy (el vector original de ca_core_news_md és de 300D)
    # Truncarem aquest vector per comparar amb les dimensions de Word2Vec
    spacy_dims_to_test = [50, 100, 150, 300] 

    for dim_spacy in spacy_dims_to_test:
        print(f"Avaluant spaCy amb embeddings truncats a {dim_spacy}D...")
        
        def get_spacy_embedding_truncated(sentence: str, target_dim: int) -> np.ndarray:
            """Obté l'embedding de spaCy (doc.vector) truncat a la dimensió desitjada"""
            doc = nlp(sentence)
            # Assegurar que el vector original té prou dimensions abans de truncar
            if doc.has_vector and len(doc.vector) >= target_dim:
                return doc.vector[:target_dim]
            elif doc.has_vector: # Si el vector és més curt que target_dim, utilitzar el vector complet
                # Opcionalment, es podria fer padding amb zeros si es volgués una mida fixa sempre
                # Però per a similitud cosinus, utilitzar el vector disponible és raonable
                # print(f"Avís: vector de spaCy ({len(doc.vector)}D) més curt que target_dim ({target_dim}D) per la frase: {sentence[:30]}...")
                return doc.vector 
            else: # Si no hi ha vector per al document (poc probable amb models md/lg)
                return np.zeros(target_dim)

        similarities_spacy = []
        true_labels_spacy = val_df['label'].values

        for _, row in val_df.iterrows():
            vec1_spacy = get_spacy_embedding_truncated(row['sentence_1'], dim_spacy)
            vec2_spacy = get_spacy_embedding_truncated(row['sentence_2'], dim_spacy)
            
            # Comprovar si els vectors són vàlids (no només zeros)
            if np.all(vec1_spacy == 0) or np.all(vec2_spacy == 0) or vec1_spacy.shape[0] == 0 or vec2_spacy.shape[0] == 0:
                sim_spacy = 0.0 # Assignar 0 si algun embedding és zero o buit
            else:
                # Assegurar que els vectors tinguin la mateixa dimensió abans de calcular cosinus
                # Això és rellevant si get_spacy_embedding_truncated retorna vectors de longitud variable
                # En aquest cas, com que sempre trunquem o retornem zeros de target_dim, haurien de coincidir
                # Però una comprovació addicional no fa mal.
                # if vec1_spacy.shape[0] != vec2_spacy.shape[0]:
                #     # print(f"Discrepància de dimensions: {vec1_spacy.shape} vs {vec2_spacy.shape}")
                #     sim_spacy = 0.0 # O una altra estratègia
                # else:
                sim_spacy = 1 - cosine(vec1_spacy, vec2_spacy)
            
            # Escalar de [-1,1] a [0,5]
            sim_scaled_spacy = (sim_spacy + 1) * 2.5 
            similarities_spacy.append(sim_scaled_spacy)

        similarities_spacy = np.array(similarities_spacy)
        # Clip per assegurar que estigui en el rang [0,5]
        similarities_spacy = np.clip(similarities_spacy, 0, 5)

        pearson_corr_spacy, _ = pearsonr(true_labels_spacy, similarities_spacy)
        mse_spacy = mean_squared_error(true_labels_spacy, similarities_spacy)
        mae_spacy = mean_absolute_error(true_labels_spacy, similarities_spacy)
        
        spacy_results_list.append({
            'Model': 'Baseline spaCy (ca_core_news_md)',
            'Dimensions': f"{dim_spacy}D (truncat)",
            'Pearson': pearson_corr_spacy,
            'MSE': mse_spacy,
            'MAE': mae_spacy
        })
        print(f"  Resultats spaCy {dim_spacy}D - Pearson: {pearson_corr_spacy:.3f}, MSE: {mse_spacy:.3f}, MAE: {mae_spacy:.3f}")


    if spacy_results_list:
        df_spacy_results = pd.DataFrame(spacy_results_list)
        df_results = pd.concat([df_results, df_spacy_results], ignore_index=True)
        print("\n--- Resultats Consolidats amb spaCy ---")
        display(df_spacy_results.style.hide(axis="index"))
    else:
        print("No s'han generat resultats per a spaCy.")

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
# import numpy as np # Ja importat
# from scipy.stats import pearsonr # Ja importat
# from sklearn.metrics import mean_squared_error, mean_absolute_error # Ja importat
# from scipy.spatial.distance import cosine # Ja importat
from tqdm.auto import tqdm # tqdm.auto per a compatibilitat notebook/consola

print("\n=== AVALUACIÓ AMB EMBEDDINGS DE ROBERTA (projecte-aina/roberta-base-ca-v2) ===")

# Comprovar disponibilitat de GPU per a PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilitzant dispositiu per a RoBERTa: {device}")

try:
    roberta_model_name = 'projecte-aina/roberta-base-ca-v2'
    roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
    roberta_model = AutoModel.from_pretrained(roberta_model_name).to(device) # Moure model al dispositiu
    roberta_available = True
except Exception as e:
    print(f"Error carregant el model RoBERTa: {e}")
    print("Saltant l'avaluació amb RoBERTa base.")
    roberta_available = False


def get_roberta_embeddings_batch_custom(sentences: List[str], tokenizer, model, 
                                   target_dim: int, batch_size: int = 16, # Reduït batch_size per si hi ha problemes de memòria
                                   max_length_tokenizer: int = 128) -> np.ndarray: # Reduït max_length per si les frases són molt llargues
    """Obté embeddings de RoBERTa per un batch de frases, utilitzant l'embedding del token [CLS]"""
    all_embeddings = []
    model.eval() # Posar el model en mode avaluació

    for i in tqdm(range(0, len(sentences), batch_size), desc="Processant batches RoBERTa"):
        batch_sentences = sentences[i:i+batch_size]
        
        inputs = tokenizer(
            batch_sentences, 
            return_tensors="pt", 
            truncation=True, 
            padding=True, # 'longest' per defecte si no s'especifica max_length
            max_length=max_length_tokenizer # Especificar max_length
        ).to(device) # Moure inputs al dispositiu
        
        with torch.no_grad():
            outputs = model(**inputs)
            # Usar l'embedding del token [CLS] (primer token) de la darrera capa oculta
            batch_cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy() # Moure a CPU i convertir a numpy
            
            # Truncar o fer padding a la dimensió desitjada
            current_dim = batch_cls_embeddings.shape[1]
            if current_dim > target_dim:
                batch_embeddings_processed = batch_cls_embeddings[:, :target_dim]
            elif current_dim < target_dim:
                # Fer padding amb zeros si l'embedding és més curt que target_dim
                padding = np.zeros((batch_cls_embeddings.shape[0], target_dim - current_dim))
                batch_embeddings_processed = np.concatenate([batch_cls_embeddings, padding], axis=1)
            else:
                batch_embeddings_processed = batch_cls_embeddings
            
            all_embeddings.append(batch_embeddings_processed)
    
    if not all_embeddings:
        # Retornar un array buit amb la forma correcta si no hi ha frases
        return np.empty((0, target_dim))
        
    return np.vstack(all_embeddings)

def calculate_cosine_similarities_vectorized_robust(embeddings1: np.ndarray, embeddings2: np.ndarray) -> np.ndarray:
    """Calcula similituds cosinus de forma vectoritzada, gestionant vectors zero."""
    # Assegurar que els embeddings no siguin buits
    if embeddings1.shape[0] == 0 or embeddings2.shape[0] == 0:
        return np.array([])

    # Normalitzar vectors
    norms1 = np.linalg.norm(embeddings1, axis=1, keepdims=True)
    norms2 = np.linalg.norm(embeddings2, axis=1, keepdims=True)
    
    # Evitar divisió per zero per a vectors que són completament zero
    # Si la norma és zero, el vector normalitzat serà zero, i el producte escalar serà zero.
    embeddings1_norm = np.divide(embeddings1, norms1, out=np.zeros_like(embeddings1), where=norms1!=0)
    embeddings2_norm = np.divide(embeddings2, norms2, out=np.zeros_like(embeddings2), where=norms2!=0)
    
    # Calcular similitud cosinus (producte escalar de vectors normalitzats)
    cosine_sims = np.sum(embeddings1_norm * embeddings2_norm, axis=1)
    
    return cosine_sims


if roberta_available:
    # Dimensions a avaluar (la dimensió original de RoBERTa base és 768)
    # Truncarem per comparar, i també utilitzarem la dimensió completa.
    roberta_dims_to_test = [50, 100, 150, 300, 768] 
    roberta_results_list = []

    # Preparar les frases del conjunt de validació
    val_sentences1 = val_df['sentence_1'].tolist()
    val_sentences2 = val_df['sentence_2'].tolist()
    val_true_labels = val_df['label'].values

    for dim_roberta in roberta_dims_to_test:
        print(f"\n--- Avaluant RoBERTa Base amb embeddings de {dim_roberta}D ---")
        
        print("Obtenint embeddings RoBERTa per a les frases 1 del conjunt de validació...")
        embeddings1_roberta = get_roberta_embeddings_batch_custom(val_sentences1, roberta_tokenizer, roberta_model, 
                                                             target_dim=dim_roberta)
        
        print("Obtenint embeddings RoBERTa per a les frases 2 del conjunt de validació...")
        embeddings2_roberta = get_roberta_embeddings_batch_custom(val_sentences2, roberta_tokenizer, roberta_model, 
                                                             target_dim=dim_roberta)
        
        if embeddings1_roberta.shape[0] > 0 and embeddings2_roberta.shape[0] > 0:
            print("Calculant similituds cosinus...")
            cosine_sims_roberta = calculate_cosine_similarities_vectorized_robust(embeddings1_roberta, embeddings2_roberta)
            
            # Escalar similituds de [-1,1] a [0,5]
            # La similitud cosinus ja està en [-1,1]. (sim + 1) -> [0,2]. (sim+1)*2.5 -> [0,5]
            similarities_scaled_roberta = (cosine_sims_roberta + 1) * 2.5
            similarities_scaled_roberta = np.clip(similarities_scaled_roberta, 0, 5) # Assegurar límits
            
            # Verificar valors problemàtics (NaN, Inf)
            if np.any(np.isnan(similarities_scaled_roberta)) or np.any(np.isinf(similarities_scaled_roberta)):
                print("Avís: S'han trobat valors NaN o Inf en les similituds escalades. S'estan substituint.")
                similarities_scaled_roberta = np.nan_to_num(similarities_scaled_roberta, nan=np.mean(val_true_labels), 
                                                            posinf=5.0, neginf=0.0)
            
            # Calcular mètriques
            pearson_corr_roberta, p_value_roberta = pearsonr(val_true_labels, similarities_scaled_roberta)
            mse_roberta = mean_squared_error(val_true_labels, similarities_scaled_roberta)
            mae_roberta = mean_absolute_error(val_true_labels, similarities_scaled_roberta)
            
            print(f"  Resultats RoBERTa {dim_roberta}D - Pearson: {pearson_corr_roberta:.3f}, MSE: {mse_roberta:.3f}, MAE: {mae_roberta:.3f}")
            
            roberta_results_list.append({
                'Model': 'Baseline RoBERTa Base (CLS, cosinus)',
                'Dimensions': f'{dim_roberta}D{" (original)" if dim_roberta == 768 else " (truncat)"}',
                'Pearson': pearson_corr_roberta,
                'MSE': mse_roberta,
                'MAE': mae_roberta
            })
        else:
            print(f"No s'han pogut generar embeddings per a RoBERTa {dim_roberta}D.")


    if roberta_results_list:
        df_roberta_results = pd.DataFrame(roberta_results_list)
        df_results = pd.concat([df_results, df_roberta_results], ignore_index=True)
        print("\n--- Resultats Consolidats amb RoBERTa Base (CLS, cosinus) ---")
        display(df_roberta_results.style.hide(axis="index"))
    else:
        print("No s'han generat resultats per a RoBERTa Base.")

    # Opcional: Netejar memòria de GPU si ja no es necessiten els models RoBERTa
    del roberta_model, roberta_tokenizer
    if device.type == 'cuda':
        torch.cuda.empty_cache()

In [ ]:
from transformers import pipeline, AutoTokenizer
# from scipy.special import logit # No sembla utilitzar-se aquí
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.auto import tqdm

print("\n=== AVALUANT AMB MODEL RoBERTa ESPECÍFICAMENT FINE-TUNED PER STS (projecte-aina/roberta-base-ca-v2-cased-sts) ===")

# Comprovar disponibilitat de GPU per a PyTorch
device_sts = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Determinar l'índex del dispositiu per a la pipeline
pipeline_device_idx = 0 if device_sts.type == "cuda" else -1 
print(f"Utilitzant dispositiu per a RoBERTa STS fine-tuned: {device_sts} (índex pipeline: {pipeline_device_idx})")


try:
    sts_model_name = 'projecte-aina/roberta-base-ca-v2-cased-sts'
    # No cal carregar el tokenizer per separat si la pipeline ho fa, però és bona pràctica per si es volgués més control
    # sts_tokenizer = AutoTokenizer.from_pretrained(sts_model_name) 
    
    # La pipeline 'text-classification' amb aquest model retorna directament un score que es pot interpretar
    # com a similitud. El model està entrenat per a STS.
    sts_pipe = pipeline('text-classification', model=sts_model_name, device=pipeline_device_idx) # Passar device a la pipeline
    sts_fine_tuned_available = True
except Exception as e:
    print(f"Error carregant el model RoBERTa STS fine-tuned: {e}")
    print("Saltant l'avaluació amb RoBERTa STS fine-tuned.")
    sts_fine_tuned_available = False


def get_sts_scores_from_pipeline(sentence_pairs_list: List[Tuple[str, str]], pipe_model, 
                                 batch_size_pipe: int = 16) -> List[float]:
    """
    Obté les puntuacions de similitud d'un model STS fine-tuned usant una pipeline.
    La pipeline per a 'text-classification' amb un model STS retorna {'label': 'LABEL_X', 'score': YYY}
    El 'score' aquí és la probabilitat de la classe predita. Per a models STS binaris (similar/no similar) o
    regressió directa, la interpretació del 'score' pot variar.
    Aquest model ('projecte-aina/roberta-base-ca-v2-cased-sts') sembla estar entrenat per a regressió
    i la seva sortida 'score' ja és una mesura de similitud (possiblement en [0,1] o un altre rang).
    Necessitem verificar com s'han d'interpretar aquests scores.
    Si el model retorna un score que ja està en l'escala [0,5] o [0,1] que es pot escalar a [0,5], perfecte.
    Si retorna logits o probabilitats per a classes discretes, caldria un post-processament.
    
    Consultant la pàgina del model a Hugging Face (si disponible) o experimentant,
    es veu que aquest model retorna un 'score' que sembla estar directament relacionat amb la similitud.
    Assumirem que aquest 'score' es pot escalar linealment si no està ja en el rang [0,5].
    La documentació del model indica que prediu valors entre 0 i 1, que després s'escalen a 0-5.
    "The model predicts values between 0 and 1. These are then scaled to 0-5."
    """
    
    # El format d'entrada per a la pipeline amb parells de frases pot variar.
    # Per a 'text-classification' amb models STS, sovint s'espera una única cadena concatenada
    # o la pipeline pot gestionar directament parells.
    # Si la pipeline espera [[sent1, sent2], [sent3, sent4], ...]:
    # formatted_inputs = [[s1, s2] for s1, s2 in sentence_pairs_list]
    
    # Si la pipeline espera [ "sent1 [SEP] sent2", "sent3 [SEP] sent4", ...]:
    # (Aquest sembla ser un format comú per a models basats en BERT/RoBERTa per a STS)
    # formatted_inputs = [f"{s1} {pipe_model.tokenizer.sep_token} {s2}" for s1, s2 in sentence_pairs_list]
    # No obstant, la pipeline de 'text-classification' sol esperar una única seqüència de text per entrada.
    # Per a STS, sovint s'utilitza 'sentence-similarity' o es passen els dos segments.
    # La pipeline 'text-classification' amb aquest model específic sembla que gestiona internament
    # la tokenització de parells si se li passen com a llista de tuples/llistes.
    
    all_scores = []
    
    # Processar en lots
    for i in tqdm(range(0, len(sentence_pairs_list), batch_size_pipe), desc="Processant RoBERTa STS fine-tuned"):
        batch_pairs = sentence_pairs_list[i:i+batch_size_pipe]
        
        # La pipeline de text-classification espera una llista de textos o llistes de parells de textos
        # Per a aquest model STS, s'espera una llista de parells:
        # [{'text': sentence1, 'text_pair': sentence2}, ...] o similar.
        # O directament: pipe(sentence_pairs_list) si sentence_pairs_list és List[Tuple[str,str]]
        
        try:
            # La pipeline per a 'text-classification' amb aquest model específic
            # potser espera els parells directament.
            predictions = pipe_model(batch_pairs) # Passar la llista de tuples (frase1, frase2)
        except Exception as e:
            # Si falla, provar amb el format concatenat (menys probable per a 'text-classification' directament)
            # print(f"Error amb input directe de parells: {e}. Provant format concatenat.")
            # formatted_batch = [f"{s1} {pipe_model.tokenizer.sep_token} {s2}" for s1, s2 in batch_pairs]
            # predictions = pipe_model(formatted_batch)
            # O potser la pipeline espera una llista de diccionaris:
            # formatted_batch = [{"text": s1, "text_pair": s2} for s1,s2 in batch_pairs]
            # predictions = pipe_model(formatted_batch)
            print(f"Error en processar el batch amb la pipeline: {e}")
            # Afegir valors placeholder si hi ha error per mantenir la longitud
            all_scores.extend([np.nan] * len(batch_pairs)) # O un valor com 2.5
            continue

        # Extreure els scores. La sortida de la pipeline és una llista de diccionaris.
        # Cada diccionari conté 'label' i 'score'.
        # Per a aquest model STS, el 'score' és el que ens interessa.
        # Aquest score està en el rang [0,1] segons la documentació del model.
        current_batch_scores = [pred['score'] for pred in predictions]
        all_scores.extend(current_batch_scores)
        
    return all_scores


if sts_fine_tuned_available:
    # Preparar parelles de frases del dataset de validació
    val_sentence_pairs_sts = [(row['sentence_1'], row['sentence_2']) for _, row in val_df.iterrows()]
    val_true_labels_sts = val_df['label'].values

    print("Obtenint prediccions del model RoBERTa STS fine-tuned...")
    # Obtenir scores bruts (assumits en [0,1])
    raw_predictions_sts = get_sts_scores_from_pipeline(val_sentence_pairs_sts, sts_pipe)
    
    # Convertir a array numpy
    raw_predictions_sts = np.array(raw_predictions_sts)

    # Escalar els scores de [0,1] a [0,5]
    scaled_predictions_sts = raw_predictions_sts * 5.0
    scaled_predictions_sts = np.clip(scaled_predictions_sts, 0, 5) # Assegurar límits

    # Verificar valors problemàtics (NaN, Inf)
    if np.any(np.isnan(scaled_predictions_sts)) or np.any(np.isinf(scaled_predictions_sts)):
        print("Avís: S'han trobat valors NaN o Inf en les prediccions STS fine-tuned. S'estan substituint.")
        # Substituir per la mitjana de les etiquetes reals si hi ha NaNs
        mean_label = np.mean(val_true_labels_sts[~np.isnan(val_true_labels_sts)]) # Mitjana de labels vàlids
        scaled_predictions_sts = np.nan_to_num(scaled_predictions_sts, nan=mean_label, 
                                                posinf=5.0, neginf=0.0)

    # Calcular mètriques
    pearson_corr_sts, p_value_sts = pearsonr(val_true_labels_sts, scaled_predictions_sts)
    mse_sts = mean_squared_error(val_true_labels_sts, scaled_predictions_sts)
    mae_sts = mean_absolute_error(val_true_labels_sts, scaled_predictions_sts)

    print(f"\n--- RESULTATS RoBERTa STS FINE-TUNED (projecte-aina/roberta-base-ca-v2-cased-sts) ---")
    print(f"Correlació de Pearson: {pearson_corr_sts:.4f} (p-value: {p_value_sts:.3g})")
    print(f"Mean Squared Error (MSE): {mse_sts:.4f}")
    print(f"Mean Absolute Error (MAE): {mae_sts:.4f}")

    # Afegir als resultats globals
    roberta_sts_fine_tuned_result = {
        'Model': 'RoBERTa STS Fine-tuned (AINA)',
        'Dimensions': 'N/A (Model Complet)', # La dimensió no és directament comparable com els embeddings truncats
        'Pearson': pearson_corr_sts,
        'MSE': mse_sts,
        'MAE': mae_sts,
    }

    df_roberta_sts_ft_results = pd.DataFrame([roberta_sts_fine_tuned_result])
    df_results = pd.concat([df_results, df_roberta_sts_ft_results], ignore_index=True)

    display(df_roberta_sts_ft_results.style.hide(axis="index"))
    
    # Netejar memòria
    del sts_pipe
    if device_sts.type == 'cuda':
        torch.cuda.empty_cache()
else:
    print("Saltada l'avaluació amb RoBERTa STS fine-tuned perquè el model no està disponible.")

## 10. Conclusions Parcials (STS) i Observacions

Després d'avaluar diversos models per a la Similitud de Text Semàntic (STS) en català, podem extreure algunes conclusions i observacions clau basades en els resultats obtinguts fins ara en el conjunt de validació.

### Resultats Principals (Esperats):

1.  **Baselines Cosinus (Word2Vec)**:
    *   Proporcionen un punt de partida raonable.
    *   L'ús de TF-IDF per ponderar els embeddings de paraules generalment millora lleugerament la mitjana simple.
    *   Dimensions més altes (ex: 300D) tendeixen a funcionar millor que les més baixes (ex: 50D) amb Word2Vec.

2.  **Model de Regressió Agregat (Word2Vec)**:
    *   Aquesta xarxa neuronal simple sol superar els baselines de similitud cosinus, demostrant la capacitat d'aprendre relacions més complexes a partir dels embeddings de frase concatenats.
    *   La tendència respecte a la dimensionalitat dels embeddings sol ser similar a la dels baselines.

3.  **Model de Seqüència amb Atenció (Word2Vec)**:
    *   **Embeddings Pre-entrenats (Frozen vs. Trainable)**:
        *   Els embeddings congelats (frozen) ja ofereixen un bon rendiment.
        *   Permetre l'entrenament (fine-tuning) dels embeddings pre-entrenats sovint condueix a millors resultats, ja que s'adapten a la tasca específica.
    *   **Embeddings Aleatoris**: Com era d'esperar, entrenar embeddings des de zero sense coneixement previ generalment resulta en un rendiment inferior, especialment amb datasets de mida limitada.
    *   Aquesta arquitectura, en general, té el potencial de superar el model agregat, ja que pot ponderar la importància de les paraules dins de les frases.

4.  **Baseline One-Hot Encoding**:
    *   Com s'esperava, aquest model simple té un rendiment significativament inferior als basats en embeddings densos, destacant la importància de les representacions semàntiques riques.

5.  **Ajustament d'Hiperparàmetres**:
    *   L'experimentació amb hiperparàmetres (mida de capes ocultes, dropout, taxa d'aprenentatge) pot portar a millores en el rendiment del model agregat. Mostra la sensibilitat del model a aquestes configuracions.

6.  **Embeddings de spaCy (`ca_core_news_md`)**:
    *   Els embeddings de document/frase de spaCy (basats en vectors de paraules contextualitzats internament) ofereixen un rendiment competitiu, sovint comparable o millor que els baselines Word2Vec simples, especialment quan s'utilitza el vector complet del document.

7.  **Embeddings de RoBERTa Base (`projecte-aina/roberta-base-ca-v2`)**:
    *   Utilitzar l'embedding del token `[CLS]` d'un model Transformer com RoBERTa com a representació de la frase, i després calcular la similitud cosinus, sol oferir resultats molt bons, superant generalment els mètodes basats en Word2Vec i spaCy. La dimensió completa (768D) sol ser la millor.

8.  **Model RoBERTa Fine-Tuned per STS (`projecte-aina/roberta-base-ca-v2-cased-sts`)**:
    *   **Aquest és, amb diferència, el model que s'espera que tingui el millor rendiment.** En estar específicament entrenat per a la tasca de STS en català, aprofita l'arquitectura potent de RoBERTa i l'especialització en la tasca. La correlació de Pearson obtinguda amb aquest model sol ser el sostre de rendiment en aquest tipus de datasets.

### Observacions Generals:

*   **Importància dels Embeddings Pre-entrenats**: L'ús d'embeddings pre-entrenats (Word2Vec, spaCy, RoBERTa) és crucial per obtenir bons resultats, especialment quan les dades d'entrenament per a la tasca específica són limitades.
*   **Complexitat del Model vs. Rendiment**: Augmentar la complexitat del model (d'un baseline a una xarxa neuronal, o d'una xarxa simple a una amb atenció o un Transformer) generalment porta a millors resultats, però també augmenta la necessitat de dades i recursos computacionals.
*   **Fine-tuning**: El fine-tuning d'embeddings pre-entrenats o de models Transformer complets en el dataset específic de la tasca sol ser la clau per assolir el màxim rendiment.
*   **Mètriques**: La correlació de Pearson és la mètrica estàndard i més important per avaluar els sistemes de STS.

A continuació, es guardaran tots els resultats i es mostrarà el millor model global identificat fins ara basant-se en la correlació de Pearson en el conjunt de validació. Després, s'avaluarà el rendiment d'aquest millor model en el conjunt de test.

## Conclusions i Observacions (Original - Es manté per context històric)

Aquesta era la secció de conclusions original del notebook. Les conclusions més actualitzades i detallades es troben en la cel·la Markdown anterior.

### Resultats Principals:

1. **Baselines Cosinus**: Proporcionen una base sòlida per comparar models més complexos
2. **Models de Regressió**: Milloren significativament sobre els baselines
3. **Impacte de les Dimensions**: Les dimensions més altes generalment milloren el rendiment
4. **TF-IDF vs Mitjana Simple**: TF-IDF sovint proporciona millors resultats
5. **Fine-tuning d'Embeddings**: Pot millorar el rendiment però amb risc d'overfitting

### Observacions:

- La correlació de Pearson és la mètrica principal per STS
- L'arquitectura amb atenció permet modelar millor les dependències entre paraules
- Els embeddings pre-entrenats proporcionen una base sòlida
- La regularització (dropout, batch normalization) és important per evitar overfitting

### Futures Direccions:

- Experimentar amb arquitectures més complexes (Transformers)
- Provar altres tècniques d'agregació (atenció multi-cap)
- Avaluar en altres tasques de NLP en català
- Combinar múltiples tipus d'embeddings

In [ ]:
# Guardar tots els resultats acumulats en un fitxer CSV
output_csv_path = 'resultats_sts_practica4_complet.csv'
try:
    df_results.to_csv(output_csv_path, index=False)
    print(f"Tots els resultats guardats a '{output_csv_path}'")
except Exception as e:
    print(f"Error guardant els resultats a CSV: {e}")

# Mostrar el millor model global basat en la correlació de Pearson en el conjunt de validació
if not df_results.empty:
    # Assegurar que la columna 'Pearson' sigui numèrica i gestionar NaNs abans de trobar idxmax
    df_results['Pearson'] = pd.to_numeric(df_results['Pearson'], errors='coerce')
    df_results_valid_pearson = df_results.dropna(subset=['Pearson'])

    if not df_results_valid_pearson.empty:
        best_model_idx = df_results_valid_pearson['Pearson'].idxmax()
        best_model_info = df_results_valid_pearson.loc[best_model_idx]

        print(f"\n🏆 MILLOR MODEL GLOBAL (basat en el conjunt de validació):")
        print(f"Model       : {best_model_info['Model']}")
        print(f"Dimensions  : {best_model_info['Dimensions']}")
        print(f"Pearson     : {best_model_info['Pearson']:.4f}")
        print(f"MSE         : {best_model_info['MSE']:.4f}")
        print(f"MAE         : {best_model_info['MAE']:.4f}")
        
        print("\n--- Taula Completa de Resultats (ordenada per Pearson descendent) ---")
        display(df_results.sort_values(by='Pearson', ascending=False).reset_index(drop=True))
    else:
        print("No s'han trobat resultats vàlids amb la mètrica Pearson per determinar el millor model.")
else:
    print("El DataFrame de resultats està buit. No es pot determinar el millor model.")


**Avaluació Final en el Conjunt de Test**

Després d'haver experimentat amb diversos models i configuracions en el conjunt de validació, i havent identificat el model amb millor rendiment (que s'espera sigui el RoBERTa fine-tuned per STS), és crucial avaluar aquest model en el conjunt de test. El conjunt de test és un conjunt de dades que el model no ha vist mai, ni directament (entrenament) ni indirectament (ajust d'hiperparàmetres o selecció de model basat en validació). Aquesta avaluació proporciona una estimació més realista de com el model generalitzarà a dades noves.

**Procés:**
1.  S'utilitza el model RoBERTa fine-tuned per STS (`projecte-aina/roberta-base-ca-v2-cased-sts`) a través de la `pipeline` de `transformers`.
2.  Es preparen els parells de frases del conjunt de test (`test_df`).
3.  Es fan prediccions de similitud per a aquests parells.
4.  Les prediccions (originalment en `[0,1]`) s'escalen a `[0,5]`.
5.  Es calculen les mètriques (Pearson, MSE, MAE) comparant les prediccions amb les etiquetes reals del conjunt de test.

Els resultats obtinguts en el conjunt de test són els que es reportarien com el rendiment final del sistema desenvolupat.

In [ ]:
# Avaluació final en test amb RoBERTa STS fine-tuned
# (Assumint que aquest és el millor model identificat)

print("=== AVALUACIÓ FINAL EN EL CONJUNT DE TEST (amb RoBERTa STS Fine-tuned) ===")

# Comprovar disponibilitat de GPU per a PyTorch
device_test = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline_device_idx_test = 0 if device_test.type == "cuda" else -1
print(f"Utilitzant dispositiu per a l'avaluació en test: {device_test} (índex pipeline: {pipeline_device_idx_test})")

try:
    sts_model_name_test = 'projecte-aina/roberta-base-ca-v2-cased-sts'
    # Recrear la pipeline per al test (bona pràctica per si s'havia eliminat)
    sts_pipe_test = pipeline('text-classification', model=sts_model_name_test, device=pipeline_device_idx_test)
    sts_ft_model_for_test_available = True
except Exception as e:
    print(f"Error carregant el model RoBERTa STS fine-tuned per al test: {e}")
    print("Saltant l'avaluació en el conjunt de test.")
    sts_ft_model_for_test_available = False


if sts_ft_model_for_test_available:
    # Preparar parelles de frases del conjunt de test
    test_sentence_pairs = [(row['sentence_1'], row['sentence_2']) for _, row in test_df.iterrows()]
    test_true_labels = test_df['label'].values

    print("Obtenint prediccions per al conjunt de test...")
    # Obtenir scores bruts (assumits en [0,1]) del model
    # Utilitzar la funció get_sts_scores_from_pipeline definida anteriorment
    raw_predictions_test = get_sts_scores_from_pipeline(test_sentence_pairs, sts_pipe_test, batch_size_pipe=16) # Ajustar batch_size si cal
    raw_predictions_test = np.array(raw_predictions_test)

    # Escalar els scores de [0,1] a [0,5]
    scaled_predictions_test = raw_predictions_test * 5.0
    scaled_predictions_test = np.clip(scaled_predictions_test, 0, 5) # Assegurar límits

    # Verificar valors problemàtics (NaN, Inf)
    if np.any(np.isnan(scaled_predictions_test)) or np.any(np.isinf(scaled_predictions_test)):
        print("Avís: S'han trobat valors NaN o Inf en les prediccions del test. S'estan substituint.")
        mean_label_test = np.mean(test_true_labels[~np.isnan(test_true_labels)])
        scaled_predictions_test = np.nan_to_num(scaled_predictions_test, nan=mean_label_test, 
                                                posinf=5.0, neginf=0.0)

    # Calcular mètriques en el conjunt de test
    pearson_corr_test, p_value_test = pearsonr(test_true_labels, scaled_predictions_test)
    mse_test = mean_squared_error(test_true_labels, scaled_predictions_test)
    mae_test = mean_absolute_error(test_true_labels, scaled_predictions_test)

    print(f"\n--- RESULTATS EN EL CONJUNT DE TEST (RoBERTa STS Fine-tuned) ---")
    print(f"Correlació de Pearson: {pearson_corr_test:.4f} (p-value: {p_value_test:.3g})")
    print(f"Mean Squared Error (MSE): {mse_test:.4f}")
    print(f"Mean Absolute Error (MAE): {mae_test:.4f}")

    # Guardar aquests resultats finals si es desitja
    test_results_summary = {
        'Model': 'RoBERTa STS Fine-tuned (AINA) - TEST SET',
        'Dimensions': 'N/A (Model Complet)',
        'Pearson': pearson_corr_test,
        'MSE': mse_test,
        'MAE': mae_test,
        'P-Value (Pearson)': p_value_test
    }
    print("\nResum dels resultats en el conjunt de test:")
    for key, value in test_results_summary.items():
        print(f"{key:<25}: {value:.4f}" if isinstance(value, float) else f"{key:<25}: {value}")
        
    # Netejar memòria
    del sts_pipe_test
    if device_test.type == 'cuda':
        torch.cuda.empty_cache()
else:
    print("No s'ha pogut realitzar l'avaluació en el conjunt de test perquè el model RoBERTa STS fine-tuned no està disponible.")


# Part 2: Classificació de Text amb el Dataset TECLA

En aquesta segona part del notebook, canviem de tasca i ens centrem en la **classificació de text**. Utilitzarem el dataset **TECLA (Text Classification for Catalan)**, també del projecte AINA. L'objectiu és classificar fragments de text en català en categories predefinides.

**Estructura d'aquesta secció:**

1.  **Càrrega i Exploració del Dataset TECLA**:
    *   Carregarem el dataset TECLA utilitzant la llibreria `datasets`.
    *   Explorarem la seva estructura, el nombre de mostres, les classes (etiquetes) presents i la seva distribució.
    *   Es mostrarà un fallback a un dataset sintètic si TECLA no es pot carregar, per permetre l'execució del codi.

2.  **Preparació de Dades i Models per a Classificació**:
    *   Adaptarem les tècniques d'embedding i modelatge vistes anteriorment (per a STS) a la nova tasca de classificació.
    *   **Model de Classificació amb Embeddings Agregats**:
        *   Es definiran funcions per preparar les dades: convertir text a un embedding de frase (mitjana simple de Word2Vec) i codificar les etiquetes de classe.
        *   Es construirà un model de xarxa neuronal similar al Model 1 de STS, però amb una capa de sortida `softmax` per a la classificació multiclase.
        *   S'entrenarà i avaluarà aquest model per a diferents dimensions d'embeddings Word2Vec.
    *   **Model de Classificació amb Seqüències d'Embeddings**:
        *   Es definiran funcions per preparar les dades: convertir text a seqüències d'índexs de paraules.
        *   Es construirà un model de xarxa neuronal que processi aquestes seqüències, utilitzant una capa d'embedding (potencialment pre-entrenada i congelada) seguida de pooling i capes denses amb sortida `softmax`.
        *   S'entrenarà i avaluarà aquest model.

3.  **Anàlisi de Resultats de Classificació**:
    *   Es compararan els resultats (principalment `accuracy` i `classification_report`) dels diferents models i configuracions.
    *   Es visualitzaran mètriques com la matriu de confusió per al millor model.

Aquesta part permetrà veure com les representacions apreses o utilitzades per a STS poden ser (o no) efectives per a una tasca diferent com la classificació de text, i com es poden adaptar les arquitectures de models.

In [ ]:
# Nou cell per TECLA Classification
print("=== PART 2: CLASSIFICACIÓ DE TEXT AMB EL DATASET TECLA ===")

from datasets import load_dataset
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical # No s'utilitza si s'usa sparse_categorical_crossentropy
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar el dataset TECLA
print("Carregant dataset TECLA...")
tecla_dataset_loaded_successfully = False
try:
    tecla_train_raw = load_dataset("projecte-aina/tecla", split="train")
    tecla_test_raw = load_dataset("projecte-aina/tecla", split="test")
    tecla_val_raw = load_dataset("projecte-aina/tecla", split="validation")
    
    # Convertir a DataFrame
    train_df_tecla = pd.DataFrame(tecla_train_raw)
    test_df_tecla = pd.DataFrame(tecla_test_raw)
    val_df_tecla = pd.DataFrame(tecla_val_raw)
    
    print(f"Dataset TECLA carregat amb èxit.")
    print(f"Mostres d'entrenament (train): {len(train_df_tecla)}")
    print(f"Mostres de prova (test): {len(test_df_tecla)}")
    print(f"Mostres de validació (validation): {len(val_df_tecla)}")
    
    # Explorar les etiquetes
    print(f"\nClasses úniques en el conjunt d'entrenament: {train_df_tecla['label'].unique()}")
    print(f"Distribució de classes en el conjunt d'entrenament:")
    print(train_df_tecla['label'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}")) # Mostrar percentatges
    
    # Mostrar exemples
    print("\nExemples del dataset TECLA (entrenament):")
    for i in range(min(3, len(train_df_tecla))): # Assegurar que no excedeixi el nombre de mostres
        print(f"  Text: {train_df_tecla.iloc[i]['text'][:150]}...") # Limitar longitud del text mostrat
        print(f"  Label: {train_df_tecla.iloc[i]['label']} (Tipus: {type(train_df_tecla.iloc[i]['label'])})")
        print("-" * 60)
    tecla_dataset_loaded_successfully = True
        
except Exception as e:
    print(f"Error carregant el dataset TECLA des de Hugging Face: {e}")
    print("Creant un dataset sintètic per a la demostració de la classificació...")
    
    # Dataset sintètic si TECLA no està disponible
    # Categories més representatives (simulant un problema de sentiment o tòpic simple)
    synthetic_texts_cat = {
        "esports": [
            "El Barça guanya la lliga amb un partit emocionant.",
            "Resultats de la jornada de futbol.",
            "El jugador estrella marca dos gols.",
            "Crònica del partit de bàsquet.",
            "Nova temporada de la Formula 1."
        ] * 40, # 200 mostres
        "cultura": [
            "Nova exposició al museu d'art contemporani.",
            "Ressenya de l'últim llibre de l'autor català.",
            "El festival de cinema presenta la seva programació.",
            "Concert de música clàssica a l'auditori.",
            "Obra de teatRecomanada per la crítica."
        ] * 40, # 200 mostres
        "economia": [
            "La borsa puja després de l'anunci del banc central.",
            "Informe sobre l'estat de l'economia global.",
            "Noves mesures fiscals aprovades pel govern.",
            "L'atur disminueix lleugerament aquest trimestre.",
            "Inversió estrangera en el sector tecnològic."
        ] * 40  # 200 mostres
    }
    
    all_synthetic_texts = []
    all_synthetic_labels = []
    for label, texts in synthetic_texts_cat.items():
        all_synthetic_texts.extend(texts)
        all_synthetic_labels.extend([label] * len(texts))
        
    # Barrejar les dades sintètiques
    from sklearn.utils import shuffle
    all_synthetic_texts, all_synthetic_labels = shuffle(all_synthetic_texts, all_synthetic_labels, random_state=42)

    # Dividir en train, val, test (aproximadament 60%, 20%, 20%)
    total_samples = len(all_synthetic_texts)
    train_end = int(total_samples * 0.6)
    val_end = int(total_samples * 0.8)

    train_df_tecla = pd.DataFrame({
        'text': all_synthetic_texts[:train_end],
        'label': all_synthetic_labels[:train_end]
    })
    val_df_tecla = pd.DataFrame({
        'text': all_synthetic_texts[train_end:val_end],
        'label': all_synthetic_labels[train_end:val_end]
    })
    test_df_tecla = pd.DataFrame({
        'text': all_synthetic_texts[val_end:],
        'label': all_synthetic_labels[val_end:]
    })
    print(f"\nDataset sintètic creat:")
    print(f"Mostres d'entrenament: {len(train_df_tecla)}")
    print(f"Mostres de validació: {len(val_df_tecla)}")
    print(f"Mostres de prova: {len(test_df_tecla)}")
    print(f"Classes (sintètiques): {train_df_tecla['label'].unique()}")
    print(f"Distribució (sintètica): \n{train_df_tecla['label'].value_counts(normalize=True)}")

# Assegurar que les columnes necessàries existeixen i no són buides
if 'text' not in train_df_tecla.columns or 'label' not in train_df_tecla.columns:
    raise ValueError("El DataFrame d'entrenament de TECLA (o sintètic) no té les columnes 'text' o 'label'.")
if train_df_tecla.empty:
    raise ValueError("El DataFrame d'entrenament de TECLA (o sintètic) està buit.")


In [ ]:
# Preparació de dades per classificació
def prepare_classification_data_aggregated(df: pd.DataFrame, embeddings_dict: Dict[str, np.ndarray], 
                                         vector_size: int, label_encoder: Optional[LabelEncoder] = None) -> Tuple[np.ndarray, np.ndarray, LabelEncoder]:
    """
    Prepara les dades per al model de classificació amb embeddings agregats (mitjana simple).
    Retorna X (embeddings de frases) i Y (etiquetes numèriques), i el LabelEncoder.
    """
    X, Y_text_labels = [], []
    
    for _, row in df.iterrows():
        text = str(row['text']) # Assegurar que el text sigui string
        label = row['label']    # L'etiqueta pot ser string o int inicialment
        
        # Obtenir embedding del text (mitjana simple)
        text_embedding = get_sentence_embedding_simple(text, embeddings_dict, vector_size)
        
        X.append(text_embedding)
        Y_text_labels.append(label) # Guardar les etiquetes originals (text o int)
    
    X = np.array(X)
    
    # Codificar etiquetes de text a numèriques
    if label_encoder is None:
        label_encoder = LabelEncoder()
        # Ajustar el LabelEncoder amb les etiquetes de text/int
        Y_encoded = label_encoder.fit_transform(Y_text_labels)
    else:
        # Transformar les etiquetes de text/int utilitzant el LabelEncoder ja ajustat
        Y_encoded = label_encoder.transform(Y_text_labels)
    
    return X, Y_encoded, label_encoder


def build_classification_model(embedding_dim: int, num_classes: int, 
                             hidden_size: int = 128, dropout_rate: float = 0.3) -> tf.keras.Model:
    """
    Construeix el model de classificació amb embeddings agregats.
    """
    input_layer = tf.keras.Input(shape=(embedding_dim,), name="input_embedding")
    
    # Capes de processament
    x = tf.keras.layers.BatchNormalization()(input_layer)
    x = tf.keras.layers.Dense(hidden_size, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    x = tf.keras.layers.Dense(hidden_size // 2, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    # Opcional: una capa més petita abans de la sortida
    # x = tf.keras.layers.Dense(hidden_size // 4, activation='relu')(x)
    # x = tf.keras.layers.Dropout(dropout_rate / 2)(x) # Menys dropout a prop de la sortida
    
    # Capa de sortida (classificació)
    # num_classes ha de ser el nombre exacte de classes úniques
    output = tf.keras.layers.Dense(num_classes, activation='softmax', name="output_classification")(x) 
    
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    
    model.compile(
        loss='sparse_categorical_crossentropy', # Perquè les etiquetes Y són enters (0, 1, 2...)
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=['accuracy'] # Mètrica principal per classificació
    )
    
    return model

# Entrenar models de classificació amb embeddings agregats (Word2Vec)
classification_results_agg = {} # Canviat el nom per evitar conflictes

# Només executar si kv_model (Word2Vec) i truncated_embeddings estan disponibles
# i si el dataset TECLA (o el seu substitut sintètic) s'ha carregat.
if kv_model is not None and truncated_embeddings and not train_df_tecla.empty:
    print("\n=== ENTRENAMENT DE MODELS DE CLASSIFICACIÓ (TECLA) AMB EMBEDDINGS AGREGATS (Word2Vec) ===")
    
    # Obtenir el LabelEncoder ajustat amb totes les etiquetes possibles (train+val+test)
    # per assegurar consistència, tot i que només s'hauria d'ajustar amb train.
    # Per simplicitat aquí, si les etiquetes són numèriques i comencen des de 0, LabelEncoder ho gestiona bé.
    # Si són strings, és crucial ajustar-lo correctament.
    
    # Ajustar LabelEncoder NOMÉS amb les dades d'entrenament per evitar data leakage
    # Primer, assegurem que les etiquetes siguin consistents (p.ex. string)
    # Això és important si les etiquetes originals del dataset TECLA són enters però no seqüencials o no comencen en 0.
    # Si són strings, LabelEncoder funciona directament.
    # Si són enters, LabelEncoder els tractarà com a classes diferents.
    
    # Crear i ajustar el LabelEncoder amb les dades d'entrenament
    # S'ha de fer abans del bucle si es vol que sigui el mateix per a totes les dimensions.
    # Però la funció prepare_classification_data_aggregated ja ho gestiona internament.
    # Aquí, el primer LabelEncoder creat (per a la primera dimensió) serà reutilitzat.
    
    shared_label_encoder = None

    for dim_cls_agg in [50, 100, 150, 300]: # Dimensions dels embeddings Word2Vec a provar
        if dim_cls_agg in truncated_embeddings:
            print(f"\n--- Entrenant Model de Classificació Agregat {dim_cls_agg}D ---")
            
            # Preparar dades d'entrenament
            X_train_cls_agg, Y_train_cls_agg, current_le = prepare_classification_data_aggregated(
                train_df_tecla, truncated_embeddings[dim_cls_agg], dim_cls_agg, shared_label_encoder
            )
            if shared_label_encoder is None: # Guardar el primer LabelEncoder creat
                shared_label_encoder = current_le

            # Preparar dades de validació utilitzant el LabelEncoder ajustat amb les dades d'entrenament
            X_val_cls_agg, Y_val_cls_agg, _ = prepare_classification_data_aggregated(
                val_df_tecla, truncated_embeddings[dim_cls_agg], dim_cls_agg, shared_label_encoder
            )
            
            num_classes_tecla = len(shared_label_encoder.classes_)
            print(f"Nombre de classes detectades pel LabelEncoder: {num_classes_tecla}")
            print(f"Classes (mapejades per LabelEncoder): {list(shared_label_encoder.classes_)}")
            # print(f"Exemple d'etiquetes codificades (train): {Y_train_cls_agg[:5]}")
            
            print(f"Forma de les dades d'entrenament: X_train={X_train_cls_agg.shape}, Y_train={Y_train_cls_agg.shape}")
            print(f"Forma de les dades de validació: X_val={X_val_cls_agg.shape}, Y_val={Y_val_cls_agg.shape}")
            
            if num_classes_tecla <= 1:
                print(f"Avís: Només s'ha detectat {num_classes_tecla} classe. La classificació no és possible o trivial. Saltant entrenament per a {dim_cls_agg}D.")
                continue

            # Construir i entrenar model
            model_cls_agg = build_classification_model(
                embedding_dim=dim_cls_agg, 
                num_classes=num_classes_tecla,
                hidden_size=256, # Potser una mica més gran per classificació
                dropout_rate=0.4
            )
            
            # Callbacks
            early_stopping_cls = tf.keras.callbacks.EarlyStopping(
                monitor='val_accuracy', 
                patience=10, # Més paciència per a accuracy
                restore_best_weights=True, 
                verbose=1,
                mode='max' # Maximitzar accuracy
            )
            
            reduce_lr_cls = tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', 
                factor=0.2, # Reducció més agressiva
                patience=5, 
                min_lr=1e-7, # Permetre learning rates més baixos
                verbose=1
            )
            
            print("Iniciant entrenament del model de classificació agregat...")
            history_cls_agg = model_cls_agg.fit(
                X_train_cls_agg, Y_train_cls_agg,
                validation_data=(X_val_cls_agg, Y_val_cls_agg),
                epochs=50, # Mantenir un nombre raonable d'èpoques
                batch_size=32,
                callbacks=[early_stopping_cls, reduce_lr_cls],
                verbose=1 # Mostrar progrés de l'entrenament
            )
            
            # Avaluació en el conjunt de validació
            print("Avaluant model en el conjunt de validació...")
            loss_val, accuracy_val = model_cls_agg.evaluate(X_val_cls_agg, Y_val_cls_agg, verbose=0)
            
            Y_pred_probs_cls_agg = model_cls_agg.predict(X_val_cls_agg)
            Y_pred_classes_cls_agg = np.argmax(Y_pred_probs_cls_agg, axis=1) # Obtenir la classe predita
            
            classification_results_agg[dim_cls_agg] = {
                'model': model_cls_agg,
                'history': history_cls_agg,
                'accuracy': accuracy_val, # Usar l'accuracy de model.evaluate
                'predictions_classes': Y_pred_classes_cls_agg,
                'label_encoder': shared_label_encoder, # Guardar el LE per a la descodificació
                'y_true_encoded': Y_val_cls_agg # Guardar les etiquetes reals codificades
            }
            
            print(f"Resultats Classificació Agregada {dim_cls_agg}D - Accuracy (validació): {accuracy_val:.4f}")
            
            # Report de classificació detallat
            print("\nClassification Report (validació):")
            # Assegurar que target_names siguin strings per al report
            target_names_str = [str(cls_name) for cls_name in shared_label_encoder.classes_]
            print(classification_report(
                Y_val_cls_agg, Y_pred_classes_cls_agg, 
                target_names=target_names_str,
                zero_division=0 # Evitar warnings si alguna classe no té prediccions/suport
            ))
else:
    print("Saltant l'entrenament de models de classificació agregats: kv_model, truncated_embeddings o train_df_tecla no disponibles.")

# Netejar la variable global si ja no es necessita fora d'aquest context específic
# del if kv_model...
# if 'shared_label_encoder' in globals() and not (kv_model is not None and truncated_embeddings and not train_df_tecla.empty):
#    del shared_label_encoder

In [ ]:
# Visualització de resultats de classificació (Models Agregats)
if classification_results_agg: # Comprovar si hi ha resultats per mostrar
    print("\n=== COMPARACIÓ MODELS DE CLASSIFICACIÓ AGREGATS (TECLA) ===")

    # Crear DataFrame amb resultats d'accuracy
    classification_data_agg_list = []
    for dim_res, results_res in classification_results_agg.items():
        classification_data_agg_list.append({
            'Model Tipus': 'Classificació Agregada (Word2Vec)',
            'Dimensions Embedding': f'{dim_res}D',
            'Accuracy Validació': results_res['accuracy']
        })

    df_classification_agg_summary = pd.DataFrame(classification_data_agg_list)
    if not df_classification_agg_summary.empty:
        display(df_classification_agg_summary.sort_values('Accuracy Validació', ascending=False).style.hide(axis="index"))
    else:
        print("No hi ha dades de resum de classificació agregada per mostrar.")

    # Trobar el millor model de classificació agregat basat en accuracy
    best_cls_agg_dim = -1
    best_cls_agg_accuracy = -1

    for dim_val, result_val in classification_results_agg.items():
        if result_val['accuracy'] > best_cls_agg_accuracy:
            best_cls_agg_accuracy = result_val['accuracy']
            best_cls_agg_dim = dim_val
            
    if best_cls_agg_dim != -1:
        best_cls_agg_model_info = classification_results_agg[best_cls_agg_dim]
        
        print(f"\n🏆 MILLOR MODEL DE CLASSIFICACIÓ AGREGADA (Word2Vec):")
        print(f"  Dimensions Embedding: {best_cls_agg_dim}D")
        print(f"  Accuracy (validació): {best_cls_agg_model_info['accuracy']:.4f}")
        
        # Matriu de confusió per al millor model agregat
        y_true_best_agg = best_cls_agg_model_info['y_true_encoded']
        y_pred_classes_best_agg = best_cls_agg_model_info['predictions_classes']
        current_le_best_agg = best_cls_agg_model_info['label_encoder']
        
        cm_agg = confusion_matrix(y_true_best_agg, y_pred_classes_best_agg)
        
        plt.figure(figsize=(10, 8)) # Ajustar mida per a més classes
        # Assegurar que les etiquetes de la matriu de confusió siguin strings
        cm_target_names_str = [str(cls_name) for cls_name in current_le_best_agg.classes_]
        sns.heatmap(cm_agg, annot=True, fmt='d', cmap='Blues',
                    xticklabels=cm_target_names_str,
                    yticklabels=cm_target_names_str)
        plt.title(f'Matriu de Confusió - Millor Model Agregat ({best_cls_agg_dim}D)')
        plt.ylabel('Etiqueta Real')
        plt.xlabel('Etiqueta Predita')
        plt.xticks(rotation=45, ha='right') # Rotar etiquetes si són llargues
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
    else:
        print("No s'ha pogut determinar el millor model de classificació agregat.")
        
    # Gràfic d'accuracy per dimensions (models agregats)
    dims_agg_plot = sorted(list(classification_results_agg.keys()))
    accuracies_agg_plot = [classification_results_agg[d]['accuracy'] for d in dims_agg_plot]
    
    if dims_agg_plot: # Només graficar si hi ha dades
        plt.figure(figsize=(10, 6))
        bars = plt.bar([f'{d}D' for d in dims_agg_plot], accuracies_agg_plot, alpha=0.75, color='cornflowerblue', edgecolor='black')
        plt.title('Accuracy (Validació) dels Models de Classificació Agregats per Dimensions')
        plt.ylabel('Accuracy')
        plt.xlabel('Dimensions dels Embeddings Word2Vec')
        plt.ylim(0, 1.05) # Ajustar límit superior per al text
        
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01, f'{yval:.3f}', ha='center', va='bottom')
            
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

else:
    print("No hi ha resultats de classificació agregada per visualitzar.")

In [ ]:
# Model de classificació amb seqüències
def prepare_classification_data_sequence(df: pd.DataFrame, word_to_idx_map: Dict[str, int], 
                                       max_len_seq: int = 32, # Renombrat per evitar conflicte amb global sequence_length
                                       label_encoder: Optional[LabelEncoder] = None) -> Tuple[np.ndarray, np.ndarray, LabelEncoder]:
    """
    Prepara les dades per al model de classificació amb seqüències.
    Retorna X_seq (seqüències d'índexs) i Y (etiquetes numèriques), i el LabelEncoder.
    """
    X_sequences, Y_text_labels_seq = [], []
    
    for _, row in df.iterrows():
        text = str(row['text']) # Assegurar que el text sigui string
        label = row['label']
        
        # Convertir text a seqüència d'índexs
        text_seq_indices = sentence_to_sequence(text, word_to_idx_map, max_len_seq) # Utilitzar sentence_to_sequence existent
        
        X_sequences.append(text_seq_indices)
        Y_text_labels_seq.append(label)
    
    X_sequences = np.array(X_sequences)
    
    # Codificar etiquetes de text a numèriques
    if label_encoder is None:
        label_encoder = LabelEncoder()
        Y_encoded_seq = label_encoder.fit_transform(Y_text_labels_seq)
    else:
        Y_encoded_seq = label_encoder.transform(Y_text_labels_seq)
    
    return X_sequences, Y_encoded_seq, label_encoder


def build_classification_model_sequence(vocab_size_cls: int, # Renombrat per evitar conflicte
                                       embedding_dim_cls: int, # Renombrat
                                       num_classes_cls: int, # Renombrat
                                       seq_len_cls: int = 32, # Renombrat
                                       pretrained_weights_matrix: Optional[np.ndarray] = None, # Renombrat
                                       train_embedding_layer: bool = False # Permetre entrenar la capa d'embedding
                                       ) -> tf.keras.Model:
    """
    Model de classificació amb seqüències d'embeddings.
    """
    input_layer_seq = tf.keras.Input(shape=(seq_len_cls,), dtype=tf.int32, name="input_sequence_classification")
    
    # Capa d'embedding
    if pretrained_weights_matrix is not None:
        # Utilitzar embeddings pre-entrenats
        embedding_layer_cls = tf.keras.layers.Embedding(
            input_dim=vocab_size_cls, # Mida del vocabulari
            output_dim=embedding_dim_cls, # Dimensió de l'embedding
            input_length=seq_len_cls,
            weights=[pretrained_weights_matrix], # Matriu d'embeddings pre-entrenats
            trainable=train_embedding_layer, # Decidir si es fa fine-tuning o no
            mask_zero=True # Ignorar padding (índex 0)
        )
    else:
        # Aprendre embeddings des de zero
        embedding_layer_cls = tf.keras.layers.Embedding(
            input_dim=vocab_size_cls,
            output_dim=embedding_dim_cls,
            input_length=seq_len_cls,
            trainable=True, # Sempre entrenable si no hi ha pre-entrenats
            mask_zero=True
        )
    
    embedded_sequences = embedding_layer_cls(input_layer_seq)
    
    # Processament de seqüència: GlobalAveragePooling1D és una opció simple i efectiva
    # Alternativament, es podria usar LSTM, GRU, o Conv1D
    x_seq = tf.keras.layers.GlobalAveragePooling1D()(embedded_sequences)
    # x_seq = tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences) # Alternativa
    
    # Capes denses per a la classificació
    x_seq = tf.keras.layers.BatchNormalization()(x_seq)
    x_seq = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x_seq)
    x_seq = tf.keras.layers.Dropout(0.4)(x_seq) # Dropout més alt pot ajudar
    
    x_seq = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x_seq)
    x_seq = tf.keras.layers.Dropout(0.4)(x_seq)
    
    # Sortida
    output_seq = tf.keras.layers.Dense(num_classes_cls, activation='softmax', name="output_classification_sequence")(x_seq)
    
    model_seq_classification = tf.keras.Model(inputs=input_layer_seq, outputs=output_seq)
    
    model_seq_classification.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # Potser provar 0.0005
        metrics=['accuracy']
    )
    
    return model_seq_classification


# Entrenar model de seqüència per classificació (TECLA)
print("\n=== ENTRENAMENT DE MODELS DE SEQÜÈNCIA PER CLASSIFICACIÓ (TECLA) ===")

classification_results_seq = {} # Resultats per als models de seqüència

# Assegurar que les variables globals necessàries (word_to_idx, vocab_size, sequence_length)
# de la part de STS estiguin disponibles i siguin correctes.
# També, kv_model i truncated_embeddings per a la matriu pre-entrenada.
# I el LabelEncoder dels models agregats (shared_label_encoder) per consistència.

if 'word_to_idx' in globals() and 'vocab_size' in globals() and 'sequence_length' in globals() and \
   kv_model is not None and truncated_embeddings and not train_df_tecla.empty and \
   'shared_label_encoder' in globals() and shared_label_encoder is not None:

    # Preparar dades de seqüència per a classificació (una sola vegada)
    # Utilitzar el vocabulari i la longitud de seqüència definits per STS per coherència
    # i el LabelEncoder dels models agregats de classificació.
    print(f"Utilitzant vocabulari de STS: {vocab_size} paraules, longitud de seqüència: {sequence_length}")
    
    X_train_cls_seq, Y_train_cls_seq, le_seq_cls_train = prepare_classification_data_sequence(
        train_df_tecla, word_to_idx, sequence_length, shared_label_encoder
    )
    X_val_cls_seq, Y_val_cls_seq, _ = prepare_classification_data_sequence(
        val_df_tecla, word_to_idx, sequence_length, shared_label_encoder
    )

    num_classes_tecla_seq = len(shared_label_encoder.classes_)
    print(f"Nombre de classes per al model de seqüència: {num_classes_tecla_seq}")
    print(f"Forma de les dades de seqüència (train): X={X_train_cls_seq.shape}, Y={Y_train_cls_seq.shape}")

    if num_classes_tecla_seq <= 1:
        print("Avís: Només s'ha detectat 1 classe o menys. Saltant entrenament de models de seqüència.")
    else:
        for dim_cls_seq in [50, 100, 150, 300]: # Dimensions dels embeddings Word2Vec a provar
            if dim_cls_seq in truncated_embeddings:
                print(f"\n--- Entrenant Model de Classificació de Seqüència {dim_cls_seq}D ---")
                
                # Preparar matriu d'embeddings pre-entrenats (Word2Vec)
                pretrained_matrix_cls = create_pretrained_embedding_matrix(
                    word_to_idx, truncated_embeddings[dim_cls_seq], dim_cls_seq
                )
                
                # Construir model de seqüència per classificació
                # Provarem amb embeddings pre-entrenats congelats (train_embedding_layer=False)
                # i després amb entrenables (train_embedding_layer=True)
                
                for trainable_emb_flag in [False, True]:
                    config_name = "FrozenEmb" if not trainable_emb_flag else "TrainableEmb"
                    print(f"  Configuració: Embeddings {config_name}")

                    model_classification_seq = build_classification_model_sequence(
                        vocab_size_cls=vocab_size, # Mida del vocabulari global
                        embedding_dim_cls=dim_cls_seq,
                        num_classes_cls=num_classes_tecla_seq,
                        seq_len_cls=sequence_length, # Longitud de seqüència global
                        pretrained_weights_matrix=pretrained_matrix_cls,
                        train_embedding_layer=trainable_emb_flag 
                    )
                    
                    # Callbacks
                    early_stopping_cls_seq = tf.keras.callbacks.EarlyStopping(
                        monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1, mode='max'
                    )
                    reduce_lr_cls_seq = tf.keras.callbacks.ReduceLROnPlateau(
                        monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1
                    )
                    
                    print(f"  Iniciant entrenament (Embeddings {config_name})...")
                    history_cls_seq = model_classification_seq.fit(
                        X_train_cls_seq, Y_train_cls_seq,
                        validation_data=(X_val_cls_seq, Y_val_cls_seq),
                        epochs=40, # Una mica menys d'èpoques per a la cerca ràpida
                        batch_size=32,
                        callbacks=[early_stopping_cls_seq, reduce_lr_cls_seq],
                        verbose=0 # Menys verbositat per a múltiples entrenaments
                    )
                    
                    # Avaluació
                    loss_val_seq, accuracy_val_seq = model_classification_seq.evaluate(X_val_cls_seq, Y_val_cls_seq, verbose=0)
                    
                    # Guardar resultats per a aquesta configuració específica
                    result_key = f"{dim_cls_seq}D_{config_name}"
                    classification_results_seq[result_key] = {
                        'model': model_classification_seq,
                        'accuracy': accuracy_val_seq,
                        'dimensions': dim_cls_seq,
                        'embedding_type': config_name
                        # Es podrien guardar prediccions, etc., si fos necessari
                    }
                    print(f"  Resultats Classificació Seqüència {dim_cls_seq}D ({config_name}) - Accuracy (validació): {accuracy_val_seq:.4f}")

else:
    print("Saltant l'entrenament de models de classificació de seqüència: "
          "Variables globals (word_to_idx, etc.), kv_model, truncated_embeddings, "
          "train_df_tecla o shared_label_encoder no disponibles o buits.")


# Comparació final de tots els models de classificació per a TECLA
print("\n=== COMPARACIÓ FINAL MODELS DE CLASSIFICACIÓ (TECLA) ===")
final_classification_results_list = []

# Afegir resultats dels models agregats
if classification_results_agg:
    for dim_agg, results_agg_val in classification_results_agg.items():
        final_classification_results_list.append({
            'Model Tipus': 'Classificació Agregada (Word2Vec)',
            'Configuració Embedding': f'{dim_agg}D',
            'Accuracy Validació': results_agg_val['accuracy']
        })

# Afegir resultats dels models de seqüència
if classification_results_seq:
    for key_seq, results_seq_val in classification_results_seq.items():
        final_classification_results_list.append({
            'Model Tipus': 'Classificació Seqüència (Word2Vec)',
            'Configuració Embedding': f"{results_seq_val['dimensions']}D - {results_seq_val['embedding_type']}",
            'Accuracy Validació': results_seq_val['accuracy']
        })

if final_classification_results_list:
    df_final_classification_summary = pd.DataFrame(final_classification_results_list)
    display(df_final_classification_summary.sort_values('Accuracy Validació', ascending=False).style.hide(axis="index"))
    
    # Guardar resultats de classificació TECLA
    try:
        df_final_classification_summary.to_csv('resultats_tecla_practica4.csv', index=False)
        print("Resultats de classificació TECLA guardats a 'resultats_tecla_practica4.csv'")
    except Exception as e:
        print(f"Error guardant els resultats de TECLA a CSV: {e}")
else:
    print("No hi ha resultats finals de classificació per mostrar o guardar.")

In [ ]:
# Cel·la final per a possibles anàlisis addicionals o neteja

print("\n=== FINAL DE LA PRÀCTICA 4 (STS + Classificació TECLA) ===")

# Es podria afegir aquí:
# 1. Avaluació del millor model de classificació TECLA en el seu conjunt de test (test_df_tecla).
#    - Preparar X_test_cls, Y_test_cls utilitzant el LabelEncoder i word_to_idx adients.
#    - Carregar el millor model de classificació guardat (o reentrenar-lo si no s'ha guardat).
#    - Fer prediccions i calcular mètriques (accuracy, classification_report, matriu de confusió) en el test set.

# 2. Neteja de memòria addicional si fos necessari (eliminar models grans, buidar cache de TensorFlow/PyTorch).
#    tf.keras.backend.clear_session() # Per a TensorFlow/Keras
#    torch.cuda.empty_cache() # Per a PyTorch (ja fet en seccions anteriors)

# 3. Resum executiu de les troballes principals de tota la pràctica.

# Exemple de com es podria avaluar el millor model de classificació en el test set:
# (Això requeriria identificar quin model va ser el millor i tenir les dades de test preparades)

# if final_classification_results_list: # Si hi ha resultats per triar el millor
#     df_summary_cls = pd.DataFrame(final_classification_results_list)
#     if not df_summary_cls.empty:
#         best_cls_config_row = df_summary_cls.sort_values('Accuracy Validació', ascending=False).iloc[0]
#         best_cls_model_type = best_cls_config_row['Model Tipus']
#         best_cls_config_emb = best_cls_config_row['Configuració Embedding']
#         print(f"\nMillor configuració de classificació TECLA (validació): {best_cls_model_type} amb {best_cls_config_emb}")

#         # Lògica per recuperar/reconstruir i avaluar el millor model en test_df_tecla...
#         # Això és complex perquè el model exacte i els seus paràmetres s'han de recuperar.
#         # Per exemple, si el millor va ser un model de seqüència:
#         # best_dim_str, best_emb_type_str = best_cls_config_emb.split('D - ') # Extreure info
#         # best_dim = int(best_dim_str)
#         # best_model_to_test = classification_results_seq[f"{best_dim}D_{best_emb_type_str}"]['model']
#         # ... preparar X_test_cls_seq, Y_test_cls_seq ...
#         # test_loss, test_accuracy = best_model_to_test.evaluate(X_test_cls_seq, Y_test_cls_seq)
#         # print(f"Accuracy en el conjunt de TEST per al millor model de TECLA: {test_accuracy:.4f}")
#         pass # Implementació pendent

print("\nRecordatori: Els resultats finals més fiables per a STS s'obtenen del conjunt de test amb el model RoBERTa STS Fine-tuned.")
print("Per a la classificació TECLA, s'hauria de realitzar una avaluació similar en el seu conjunt de test.")
